# ------------- Mohammad Amin Kiani 4003613052 -------------
## Final Project _ Emotion-Based Music Generation _ ui.ac.ir 400-404

# تحلیل متن    

In [ ]:
!pip install -q transformers datasets sentencepiece scikit-learn langdetect accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 55.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score, f1_score
from langdetect import detect
import torch
import numpy as np

## فاین تیون کردن

### متن فارسی

In [ ]:
!git clone https://github.com/Arman-Rayan-Sharif/arman-text-emotion.git
!ls arman-text-emotion/dataset

Cloning into 'arman-text-emotion'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 52 (delta 10), reused 9 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (52/52), 572.39 KiB | 1.84 MiB/s, done.
Resolving deltas: 100% (10/10), done.
test.tsv  train.tsv


In [ ]:
import pandas as pd

# فایل‌ها هدر ندارند
train_df = pd.read_csv(
    "arman-text-emotion/dataset/train.tsv",
    sep="\t",
    header=None,
    names=["text","label"]
)
test_df = pd.read_csv(
    "arman-text-emotion/dataset/test.tsv",
    sep="\t",
    header=None,
    names=["text","label"]
)

# حذف ردیف‌های ناقص (در صورت وجود)
train_df.dropna(subset=["text","label"], inplace=True)
test_df.dropna(subset=["text","label"], inplace=True)

print("Train sample:")
print(train_df.head(), "\n")
print("Test sample:")
print(test_df.head())


Train sample:
                                                text  label
0  خیلی کوچیک هستن و سایزشون بدرد نمیخوره میخوام ...    SAD
1     از صدای پرنده دم دمای صبح متنفرم متنفرم متنفرم   HATE
2   "کیفیتش خیلی خوبه با شک خریدم ولی واقعا راضیم...    SAD
3  چون همش با دوربین ثبت شده ، ایا میشه اعتراض زد...  OTHER
4              اين وضع ب طرز خنده داري گريه داره ...    SAD 

Test sample:
                                                text  label
0  اين شايد اولين عزاى عمومى واقعى است كه ياد دار...    SAD
1  دیشب بعد از ارسال تویت مربوط به آثار باستانی ت...  HAPPY
2   کدوم شعبه پول نداده، بگو الان برات آمار دقیق ...  OTHER
3  امروز وسط یه بحث با بابا مامانم گفتم آدم باید ...  HAPPY
4  امشب گفت نامزدی دوستش که ادم روشنفکری است بهم ...    SAD


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

# دیگر از ما نخواهد : API Key
# از لاگ‌گیری وزن جلوگیری کند : Weights & Biases (wandb)

In [ ]:
!pip install -q --upgrade transformers

import pandas as pd
import glob
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
import numpy as np
import torch
from sklearn.metrics import accuracy_score, f1_score

# 2. بارگذاری داده‌های TSV  +  با نام‌گذاری ستون‌ها
train_df = pd.read_csv(
    "arman-text-emotion/dataset/train.tsv",
    sep="\t", header=None, names=["text","label"]
)
test_df = pd.read_csv(
    "arman-text-emotion/dataset/test.tsv",
    sep="\t", header=None, names=["text","label"]
)
train_df.dropna(subset=["text","label"], inplace=True)
test_df.dropna(subset=["text","label"], inplace=True)

# 3. ساخت DatasetDict
dataset_fa = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test":  Dataset.from_pandas(test_df)
})

# 4. نگاشت برچسب‌های فارسی به عدد و زیر کلید 'labels'
label_list = sorted(train_df["label"].unique().tolist())
label2id   = {l:i for i,l in enumerate(label_list)}
id2label   = {i:l for l,i in label2id.items()}

def encode_label(ex):
    return {"labels": label2id[ex["label"]]}

#  فقط ستون لیبل حذف شود 'label'
dataset_fa = dataset_fa.map(encode_label, remove_columns=["label"])

# 5. بارگذاری توکنایزر و مدل (ParsBERT)
fa_model_name = "HooshvareLab/bert-fa-base-uncased"
fa_tokenizer  = AutoTokenizer.from_pretrained(fa_model_name)
fa_model      = AutoModelForSequenceClassification.from_pretrained(
    fa_model_name,
    num_labels=len(label_list)
)

# 6. توکن‌سازی
def tokenize_fn(ex):
    return fa_tokenizer(
        ex["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

dataset_fa = dataset_fa.map(tokenize_fn, batched=True, remove_columns=["text"])

# 7. تنظیم فرمت PyTorch
dataset_fa.set_format(
    type="torch",
    columns=["input_ids","attention_mask","labels"]
)

# 8. DataCollator
data_collator = DataCollatorWithPadding(tokenizer=fa_tokenizer)

# 9. متریک‌ها
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1":       f1_score(labels, preds, average="macro")
    }

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./parsbert-emotion-fa",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,

    # لاگ‌گیری
    logging_dir="./logs",
    logging_steps=100,    # هر ۱۰۰ step یک لاگ

    # فعال‌سازی ارزیابی و ذخیره‌سازی در طول آموزش
    do_train=True,
    do_eval=True,
    eval_steps=500,       # هر ۵۰۰ step ارزیابی
    save_steps=500,       # step برای هر 500 تا checkpoint ذخیره
    save_total_limit=2,   # حداکثر ۲ checkpoint نگه داشتن

    # (حذف load_best_model_at_end چون نیازمند evaluation_strategy )
)

trainer_fa = Trainer(
    model=fa_model,
    args=training_args,
    train_dataset=dataset_fa["train"],
    eval_dataset=dataset_fa["test"],
    tokenizer=fa_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer_fa.train()


# 13. ذخیره مدل
fa_model.save_pretrained("./final_parsbert_emotion")
fa_tokenizer.save_pretrained("./final_parsbert_emotion")
print(" مدل فارسی فاین‌تیون شد و ذخیره شد.")


Map:   0%|          | 0/6125 [00:00<?, ? examples/s]

Map:   0%|          | 0/1151 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

Map:   0%|          | 0/6125 [00:00<?, ? examples/s]

Map:   0%|          | 0/1151 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1923072073.py:103: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_fa = Trainer(


Step,Training Loss
100,1.742200
200,1.263600
300,1.029100
400,0.939300
500,0.714300
600,0.746300
700,0.751700
800,0.589300
900,0.473800
1000,0.474700


 مدل فارسی فاین‌تیون شد و ذخیره شد.


In [ ]:
print("All labels:", label_list)
print("Mapping id2label:", id2label)


All labels: ['ANGRY', 'FEAR', 'HAPPY', 'HATE', 'OTHER', 'SAD', 'SURPRISE']
Mapping id2label: {0: 'ANGRY', 1: 'FEAR', 2: 'HAPPY', 3: 'HATE', 4: 'OTHER', 5: 'SAD', 6: 'SURPRISE'}


In [ ]:
from transformers import AutoConfig

# ۱. بارگذاری کانفیگ و افزودن نگاشت
config = AutoConfig.from_pretrained(model_dir)
config.id2label   = id2label
config.label2id   = label2id

# ۲. بارگذاری مدل با کانفیگ سفارشی
model = AutoModelForSequenceClassification.from_pretrained(model_dir, config=config)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# ۳. ساخت pipeline
clf = pipeline("text-classification", model=model, tokenizer=tokenizer)

# ۴. تست
print(clf("خیلی ناراحتم و غمگینم"))


Device set to use cuda:0


[{'label': 'SAD', 'score': 0.9119796752929688}]


### متن انگلیسی

In [ ]:
import numpy as np
import torch
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    pipeline
)
from sklearn.metrics import f1_score, precision_score, recall_score


In [ ]:
# GoEmotions دیتاستِ تشخیص احساس چندجسمانی ۲۷ برچسبه

raw_en = load_dataset("go_emotions")

# تقسیم train/validation/test
print(dataset_en)

# برچسب‌ها (نام‌های ۲۷کلاسه)
label_names = dataset_en["train"].features["labels"].feature.names
print("Labels:", label_names)
num_labels = len(label_names)


DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5427
    })
})
Labels: ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [ ]:
model_name = "roberta-base"  #مدل انگلیسی دلخواه
tokenizer_en = AutoTokenizer.from_pretrained(model_name)

model_en = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    problem_type="multi_label_classification"
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
from sklearn.metrics import f1_score, precision_score, recall_score

# 1) بارگذاری دیتاست خام
raw = load_dataset("go_emotions")
dataset_en = DatasetDict({
    "train":      raw["train"],
    "validation": raw["validation"],
    "test":       raw["test"]
})

# 2) آماده‌سازی توکنایزر و مدل
tokenizer_en = AutoTokenizer.from_pretrained("roberta-base")
model_en  = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=len(raw["train"].features["labels"].feature.names),
    problem_type="multi_label_classification"
)
num_labels = model_en.config.num_labels

# 3) مستقیماً لیبل‌های اعشاری فلوت می‌سازد
def preprocess(ex):
    enc = tokenizer_en(
        ex["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    label_vec = [0.0] * num_labels
    for idx in ex["labels"]:
        label_vec[idx] = 1.0
    enc["labels"] = label_vec
    return enc

# 4) توکنایز + حذف ستون‌های زائد
dataset_en = dataset_en.map(
    preprocess,
    batched=False,
    remove_columns=["text", "id"]
)

# 5) قالب‌دهی مستقیم برای PyTorch
dataset_en.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)

# Trainer و training_args مثل قبل:
data_collator_en = DataCollatorWithPadding(tokenizer=tokenizer_en)

def compute_metrics_en(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits))
    preds = (probs.numpy() >= 0.5).astype(int)
    labels = labels.numpy()
    return {
        "f1":        f1_score(labels, preds, average="macro", zero_division=0),
        "precision": precision_score(labels, preds, average="macro", zero_division=0),
        "recall":    recall_score(labels, preds, average="macro", zero_division=0),
    }

training_args_en = TrainingArguments(
    output_dir="./roberta-go_emotions",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs_go",
    logging_steps=500,
    do_train=True,
    do_eval=True,
    eval_steps=1000,
    save_steps=1000,
    save_total_limit=2,
    report_to="none",
    run_name=None,
)

import torch
import torch.nn as nn
from transformers import Trainer

loss_fn = nn.BCEWithLogitsLoss()

class MultiLabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs["labels"].float()
        outputs = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
        )
        logits = outputs.logits
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

trainer_en = MultiLabelTrainer(
    model=model_en,
    args=training_args_en,
    train_dataset=dataset_en["train"],
    eval_dataset=dataset_en["validation"],
    tokenizer=tokenizer_en,
    data_collator=data_collator_en,
    compute_metrics=compute_metrics_en,
)


trainer_en.train()
trainer_en.save_model("./final_go_emotions")
tokenizer_en.save_pretrained("./final_go_emotions")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

/tmp/ipython-input-1721164868.py:103: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MultiLabelTrainer.__init__`. Use `processing_class` instead.
  trainer_en = MultiLabelTrainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss
500,0.179200
1000,0.117400
1500,0.105300
2000,0.099000
2500,0.095200
3000,0.089600
3500,0.084900
4000,0.085400
4500,0.084800
5000,0.081800


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.1

('./final_go_emotions/tokenizer_config.json',
 './final_go_emotions/special_tokens_map.json',
 './final_go_emotions/vocab.json',
 './final_go_emotions/merges.txt',
 './final_go_emotions/added_tokens.json',
 './final_go_emotions/tokenizer.json')

In [ ]:
# بارگذاری مدل فاین‌تیون‌شده
tokenizer_en = AutoTokenizer.from_pretrained("./final_go_emotions")
model_en     = AutoModelForSequenceClassification.from_pretrained("./final_go_emotions")

# ارزیابی احتمالات چندتایی
clf_en = pipeline(
    "text-classification",
    model=model_en,
    tokenizer=tokenizer_en,
    function_to_apply="sigmoid",
    top_k=None  # همه برچسب‌ها را برمی‌گرداند
)

# تست
print(clf_en("I am feeling very excited and joyful today!"))
print(clf_en("I am sad and depressed."))


Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[[{'label': 'LABEL_13', 'score': 0.7790734767913818}, {'label': 'LABEL_17', 'score': 0.21510884165763855}, {'label': 'LABEL_27', 'score': 0.05265502259135246}, {'label': 'LABEL_0', 'score': 0.0437777116894722}, {'label': 'LABEL_26', 'score': 0.04254526272416115}, {'label': 'LABEL_7', 'score': 0.036241091787815094}, {'label': 'LABEL_8', 'score': 0.027362694963812828}, {'label': 'LABEL_4', 'score': 0.019663216546177864}, {'label': 'LABEL_18', 'score': 0.019117606803774834}, {'label': 'LABEL_20', 'score': 0.015489795245230198}, {'label': 'LABEL_22', 'score': 0.013442710041999817}, {'label': 'LABEL_1', 'score': 0.012346770614385605}, {'label': 'LABEL_15', 'score': 0.01202961802482605}, {'label': 'LABEL_14', 'score': 0.011980804614722729}, {'label': 'LABEL_21', 'score': 0.011965765617787838}, {'label': 'LABEL_3', 'score': 0.011479067616164684}, {'label': 'LABEL_2', 'score': 0.008294496685266495}, {'label': 'LABEL_25', 'score': 0.008273624815046787}, {'label': 'LABEL_6', 'score': 0.007423783

In [ ]:
from transformers import pipeline

clf_en = pipeline(
    "text-classification",
    model="nateraw/bert-base-uncased-emotion",
    return_all_scores=False
)

print(clf_en("I feel really happy today!"))
# [{'label': 'joy', 'score': 0.99}]


Device set to use cuda:0


[{'label': 'joy', 'score': 0.9966766834259033}]


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


#تحلیل تصویر

## فاین تیون دستی

In [ ]:
# ───  ۱: کلون دیتاست FER-2013 ────────────────────────────────
!git clone https://huggingface.co/datasets/Jeneral/fer-2013


Cloning into 'fer-2013'...
remote: Enumerating objects: 18, done.
remote: Total 18 (delta 0), reused 0 (delta 0), pack-reused 18 (from 1)
Unpacking objects: 100% (18/18), 2.78 KiB | 568.00 KiB/s, done.
Filtering content: 100% (2/2), 55.17 MiB | 19.06 MiB/s, done.


In [ ]:
# ───  ۲: تبدیل .pt → DatasetDict + نگاشت لیبل‌ها ─────────────────

import torch, pickle
from datasets import Dataset, DatasetDict

# ۱) بارگذاری لیست نمونه‌ها با pickle
with open("fer-2013/train.pt","rb") as f:
    train_list = pickle.load(f)
with open("fer-2013/test.pt","rb") as f:
    test_list  = pickle.load(f)

# ۲) تبدیل لیست به Dataset
train_ds = Dataset.from_list(train_list)
test_ds  = Dataset.from_list(test_list)

# ۳)  DatasetDict (قبل از هیچ مپ دیگری)
ds = DatasetDict({
    "train": train_ds,
    "test":  test_ds
})

# ۴) نگاشت برچسب‌های رشته‌ای به عدد
label_list = ["angry","disgust","fear","happy","neutral","sad","surprise"]
label2id   = {l:i for i,l in enumerate(label_list)}
id2label   = {i:l for i,l in enumerate(label_list)}

def label_str_to_int(example):
    return {"labels": label2id[example["labels"]]}

ds = ds.map(label_str_to_int, batched=False)

# ۵) دیکد کردن بایت‌ها به تصویر PIL
from PIL import Image
import io

def decode_bytes(example):
    example["image"] = Image.open(io.BytesIO(example["img_bytes"])).convert("RGB")
    return example

ds = ds.map(decode_bytes, remove_columns=["img_bytes"])

print(ds)


# ───  ۳: دیکد کردن بایت‌ها به تصویر PIL ────────────────────────

print(ds["train"][0])
# {'labels': 0, 'image': <PIL.Image.Image image mode=RGB size=48x48>}

# ───  ۴: FeatureExtractor و مدل ViT ───────────────────────────

# ─── Config و FeatureExtractor ──────────────────────
import torch
from transformers import AutoConfig, AutoFeatureExtractor, AutoModelForImageClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ۷ کلاس
label_list = ["angry","disgust","fear","happy","neutral","sad","surprise"]
label2id   = {l:i for i,l in enumerate(label_list)}
id2label   = {i:l for i,l in enumerate(label_list)}

# ۴.۱) Config
config = AutoConfig.from_pretrained(
    "WinKawaks/vit-tiny-patch16-224",
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

# ۴.۲) FeatureExtractor
feature_extractor = AutoFeatureExtractor.from_pretrained(
    "WinKawaks/vit-tiny-patch16-224",
    size=224
)

# ۴.۳) لود مدل
model = AutoModelForImageClassification.from_pretrained(
    "WinKawaks/vit-tiny-patch16-224",
    config=config,
    ignore_mismatched_sizes=True
).to(device)

# ۴.۴) فریز کردن همه لایه‌ها به جز head
for name, param in model.named_parameters():
    if not name.startswith("classifier"):
        param.requires_grad = False

print("Trainable params:", sum(p.numel() for p in model.parameters() if p.requires_grad))


# ─── سلول ۵: پیش‌پردازش دسته‌ای و قالب‌دهی ────────────────────────
def preprocess_batch(batch):
    #  همزمان لیست تصاویر را resize+normalize
    enc = feature_extractor(images=batch["image"], return_tensors="pt")
    enc["labels"] = torch.tensor(batch["labels"], dtype=torch.long)
    return enc

# اعمال به کل دیتاست
ds = ds.map(preprocess_batch, batched=True, remove_columns=["image"])
ds.set_format(type="torch", columns=["pixel_values","labels"])


# ───  ۶:  Trainer  شروع آموزش ────────────────────────────
# معیار دقت
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=-1)
    return metric.compute(predictions=preds, references=p.label_ids)

training_args = TrainingArguments(
    output_dir="./face_emotion_vit",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,

    per_device_train_batch_size=4,       # ← خیلی کم
    per_device_eval_batch_size=4,        # ← خیلی کم

    gradient_accumulation_steps=4,       # ← effective batch = 4×4 =16
    gradient_checkpointing=True,         # ← reduce memory for activations

    learning_rate=2e-4,                  # ← اندکی بیشتر برای head
    weight_decay=0.0,

    load_best_model_at_end=True,
    metric_for_best_model="accuracy",

    fp16=torch.cuda.is_available(),      # ← mixed precision
    dataloader_num_workers=2,
    logging_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.save_model("./face_emotion_vit")
feature_extractor.save_pretrained("./face_emotion_vit")


# ───  ۷: تست نهایی (inference) ────────────────────────────────
from transformers import pipeline
pipe = pipeline(
    "image-classification",
    model="./face_emotion_vit",
    feature_extractor="./face_emotion_vit",
    device=0
)
from PIL import Image
print(pipe(Image.open("face4.jpg"), top_k=3))


Map:   0%|          | 0/28709 [00:00<?, ? examples/s]

Map:   0%|          | 0/7178 [00:00<?, ? examples/s]

Map:   0%|          | 0/28709 [00:00<?, ? examples/s]

Map:   0%|          | 0/7178 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'image'],
        num_rows: 28709
    })
    test: Dataset({
        features: ['labels', 'image'],
        num_rows: 7178
    })
})
{'labels': 0, 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=48x48 at 0x7E144DBEDED0>}
Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size

Trainable params: 1351


Map:   0%|          | 0/28709 [00:00<?, ? examples/s]

به علت محدودیت های اجرایی برای تصویر و صوت نمیتوان فاین تیون انجام داد و صرفا پایپ لاین انجام می شود...

# تحلیل صوت

In [ ]:
from transformers import pipeline
from pydub import AudioSegment

# لود مدل تحلیل احساس از صوت
model_id = "superb/wav2vec2-base-superb-er"
emotion_pipeline = pipeline("audio-classification", model=model_id)

# تبدیل mp3 ==> wav
audio = AudioSegment.from_file("/content/input2.wav") #shad
audio = audio.set_frame_rate(16000).set_channels(1)
audio.export("/content/sample.wav", format="wav")

# تست مدل روی فایل صوتی wav
audio_path = "/content/sample.wav"
results = emotion_pipeline(audio_path)
print(results)

Device set to use cuda:0


[{'score': 0.8742848038673401, 'label': 'hap'}, {'score': 0.11109784245491028, 'label': 'sad'}, {'score': 0.014354194514453411, 'label': 'neu'}, {'score': 0.00026313262060284615, 'label': 'ang'}]


# تحلیل احساسات مولتی مودال

In [ ]:
# ───  ۱: نصب کتابخانه‌های مورد نیاز ───────────────────────────────----------------------------------------------------
!pip install -q gradio transformers[sentencepiece] soundfile pillow

# ───  ۲: بارگذاری و آماده‌سازی مدل‌ها ────────────────────────────-------------------------------------------------------
import gradio as gr
from transformers import pipeline
from langdetect import detect
import soundfile as sf
import tempfile

# تعریف ایموجی و فضای ۶ کلاسه
EMOJI = {
    "ANGRY":"😠", "FEAR":"😨", "HAPPY":"😃",
    "SAD":"😢",   "SURPRISE":"😲","OTHER":"😐"
}

# ── Pipeline متن فارسی ──
pipe_fa = pipeline(
    "text-classification",
    model="/content/final_parsbert_emotion",
    tokenizer="/content/final_parsbert_emotion",
    device=0
)

# ── Pipeline متن انگلیسی single-label ──
pipe_en = pipeline(
    "text-classification",
    model="nateraw/bert-base-uncased-emotion",
    tokenizer="nateraw/bert-base-uncased-emotion",
    device=0,
    return_all_scores=False
)

def analyze_text(text, threshold):
    # تشخیص زبان
    lang = detect(text)
    if lang == "fa":
        out = pipe_fa(text)[0]
        idx = int(out["label"].split("_")[-1])
        # نگاشت HATE→ANGRY
        FA = ['ANGRY','FEAR','HAPPY','HATE','OTHER','SAD','SURPRISE']
        label = FA[idx] if FA[idx]!="HATE" else "ANGRY"
        score = out["score"]
    else:
        out = pipe_en(text)[0]
        raw = out["label"].lower()
        # نگاشت برچسب‌های انگلیسی به فضای مشترک
        MAP = {
            "joy":"HAPPY","love":"HAPPY",
            "sadness":"SAD","anger":"ANGRY",
            "fear":"FEAR","surprise":"SURPRISE"
        }
        label = MAP.get(raw, "OTHER")
        score = out["score"]

    if score < threshold:
        label = "OTHER"
    return label, round(score,3), EMOJI[label]



# —— Pipeline ابزار FER برای تصویر ——
!wget https://github.com/oarriaga/face_classification/raw/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5 -O emotion_model.h5

#  ۰: مطمئن شدن از نصب tensorflow و opencv
!pip install -q tensorflow opencv-python-headless

import numpy as np
import cv2
from tensorflow.keras.models import load_model

emotion_cnn = load_model("emotion_model.h5", compile=False)
FER_LABELS = ['angry','disgust','fear','happy','sad','surprise','neutral']
IMG_MAP = {
  "angry":"ANGRY","disgust":"ANGRY","fear":"FEAR",
  "happy":"HAPPY","sad":"SAD","surprise":"SURPRISE",
  "neutral":"OTHER"
}

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)

def analyze_image(img, threshold):
    try:
        gray = cv2.cvtColor(np.array(img.convert("RGB")), cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        if len(faces)==0:
            # کراپ از وسط وقتی چهره پیدا نشد
            h,w = gray.shape
            sz = min(h,w)//2
            x = (w-sz)//2; y = (h-sz)//2
            roi = gray[y:y+sz, x:x+sz]
        else:
            x,y,w,h = faces[0]
            roi = gray[y:y+h, x:x+w]

        face = cv2.resize(roi, (64,64), interpolation=cv2.INTER_AREA)
        face = face.astype("float32")/255.0
        face = face.reshape(1,64,64,1)

        preds = emotion_cnn.predict(face, verbose=0)[0]
        idx   = int(np.argmax(preds))
        score = float(preds[idx])
        raw   = FER_LABELS[idx]
        label = IMG_MAP.get(raw, "OTHER")

        # بدون صفر کردن با threshold
        return label, round(score,3), EMOJI[label]
    except Exception:
        return "OTHER", 0.0, EMOJI["OTHER"]


# ── Pipeline صوت (Superb) ──
audio_pipe = pipeline(
    "audio-classification",
    model="superb/wav2vec2-base-superb-er",
    device=0,
    top_k=1
)
def analyze_audio(filepath, threshold):
    # ممکن است فرمت متفاوت باشد
    arr, sr = sf.read(filepath)
    tmp = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    sf.write(tmp.name, arr, sr)
    out = audio_pipe(tmp.name)[0]
    raw = out["label"].lower()
    score = out["score"]
    AUD_MAP = {
      "hap":"HAPPY","sad":"SAD","ang":"ANGRY",
      "fea":"FEAR","sur":"SURPRISE","neu":"OTHER"
    }
    label = AUD_MAP.get(raw[:3], "OTHER")
    if score < threshold:
        label = "OTHER"
    return label, round(float(score),3), EMOJI[label]


# ───  ۳: تابع چندمودال برای Gradio ──────────────────────────────--------------------------------------------------------------
def analyze_all(text, image, audio_path, threshold):
    if text and text.strip() != "":
        return analyze_text(text, threshold)
    if image is not None:
        return analyze_image(image, threshold)
    if audio_path:
        return analyze_audio(audio_path, threshold)
    return "OTHER", 0.0, EMOJI["OTHER"]


# ───  ۴: ساخت رابط Gradio ───────────────────────────────────────----------------------------------------------------------------
with gr.Blocks(css="""
    #root {max-width:600px;margin:auto;padding:20px;}
    .out {font-size:1.3rem;text-align:center;}
""") as demo:
    gr.Markdown("## دستیار چند‌مودال تشخیص احساس")
    gr.Markdown("متن، تصویر یا صوت آپلود کنید تا یکی از ۶ کلاس اصلی را همراه درصد اطمینان و ایموجی ببینید.")
    with gr.Row():
        txt = gr.Textbox(label="متن (فارسی/انگلیسی)", lines=2, placeholder="مثال: من خیلی خوشحالم!")
        img = gr.Image(label="تصویر", type="pil")
        aud = gr.Audio(label="صوت (wav/mp3)", type="filepath")
    thr = gr.Slider(0,1,0.7,step=0.01, label="آستانه OTHER")
    btn = gr.Button("تحلیل کن", variant="primary")
    with gr.Row():
        o1 = gr.Textbox(label="احساس",    interactive=False, elem_classes="out")
        o2 = gr.Textbox(label="اعتماد",    interactive=False, elem_classes="out")
        o3 = gr.Textbox(label="ایموجی",    interactive=False, elem_classes="out")
    btn.click(analyze_all, [txt, img, aud, thr], [o1, o2, o3])

demo.launch(share=False)


Device set to use cpu
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Device set to use cpu


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

# تولید موسیقی از احساس تحلیل شده

### 1- گرادیو | فرانت در کولب

In [ ]:
# =======================  همگی باهم ==========================

# FluidSynth : ابزاری برای رندر کردن MIDI ==> به صدا

!apt-get -qq -y install fluidsynth ffmpeg # -y : یعنی بدون پرسیدن تأیید نصب  |  -qq :خروجی را ساکت می‌کند
!pip install -q gradio transformers[sentencepiece] soundfile pillow pyFluidSynth==1.3.3 \
                pretty_midi music21 librosa midi2audio langdetect tensorflow opencv-python-headless pydub

# transformers[sentencepiece] : برای مدل‌های HuggingFace

# gradio : برای رابط وب demo For machine learning model with a friendly web interface

# pillow : برای کار با تصاویر

# music21 : ابزار تئوری موسیقی

# librosa : پردازش صوت

# ----------------------------------ساوند فونت های لازم---------------------------------------------------------------------------------------------------------------------------------

# .sf2 : فرمت فایل صوتی  برای ذخیره سازی نمونه‌های صوتی

# SoundFont پیش‌فرض (General MIDI با کیفیت خوب)
!wget -q https://github.com/pepaslabs/fluidsynth-soundfonts/raw/master/FluidR3_GM/FluidR3_GM.sf2 -O /content/FluidR3_GM.sf2 \
  || wget -q https://github.com/urish/cyberbotics/raw/master/projects/robotbenchmark/resources/fluidr3/FluidR3_GM.sf2 -O /content/FluidR3_GM.sf2

# پوشه
!mkdir -p /content/sf2

# FluidR3 (بک آپ)
!wget -q https://github.com/pepaslabs/fluidsynth-soundfonts/raw/master/FluidR3_GM/FluidR3_GM.sf2 -O /content/sf2/FluidR3_GM.sf2 || true

# GM های باکیفیت از GitHub
!wget -c https://raw.githubusercontent.com/bratpeki/soundfonts/master/GeneralUser.sf2 -O /content/sf2/GeneralUser.sf2
!wget -c https://raw.githubusercontent.com/bratpeki/soundfonts/master/ChoriumRevA.sf2 -O /content/sf2/ChoriumRevA.sf2
!wget -c https://raw.githubusercontent.com/bratpeki/soundfonts/master/WeedsGM3.sf2 -O /content/sf2/WeedsGM3.sf2

# Arachno (پاپ/راک)
!wget -c "http://maxime.abbey.free.fr/mirror/arachnosoft/files/soundfonts/arachno-soundfont-10-sf2.zip" -O /content/sf2/Arachno.zip
!unzip -o /content/sf2/Arachno.zip -d /content/sf2

# برای فارسی
!wget -c "https://musical-artifacts.com/artifacts/940/Persa.sf2" -O /content/sf2/Persa.sf2

# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# مدل FER (mini_XCEPTION) برای تصویر
!wget -q https://raw.githubusercontent.com/oarriaga/face_classification/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5 -O /content/emotion_model.h5

# ------------------ Import & Config (TF → CPU) ----------------
import os, time, random, json
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

import tensorflow as tf
try:
    tf.config.set_visible_devices([], 'GPU')
except Exception:
    pass

import numpy as np
import gradio as gr
import soundfile as sf
from PIL import Image

import torch
from transformers import pipeline
import cv2
from tensorflow.keras.models import load_model
import pretty_midi
from midi2audio import FluidSynth
from langdetect import detect
from pydub import AudioSegment  # برای MP3

# ----------------------- Device برای Transformers ----------------------

DEVICE = 0 if torch.cuda.is_available() else -1
print("Device for Transformers:", "cuda:0" if DEVICE==0 else "cpu")

# SF2_PATH_DEFAULT = "/content/FluidR3_GM.sf2" اگر هیچ ساوند فونتی نداشتیم، این استاندارد دیفالته

SF2_BANKS = {
    "fluidr3": "/content/sf2/FluidR3_GM.sf2",
    "generaluser": "/content/sf2/GeneralUser.sf2",
    "chorium": "/content/sf2/ChoriumRevA.sf2",
    "weeds": "/content/sf2/WeedsGM3.sf2",
    "arachno": "/content/sf2/Arachno SoundFont - Version 1.0.sf2",
    "persa": "/content/sf2/Persa.sf2",
}

# نمونه استفاده با یکی از اعضای بانک ساوند فونت ها:
SF2_PATH_DEFAULT = SF2_BANKS["persa"]

# ---------- Utility: نرمال‌سازی/لیمیتِ خروجی برای پخش پایدار ----------
def _normalize_limit(x, peak=0.98):
    x = np.array(x)
    if x.ndim == 1:
        x = x[:, None]
    x = np.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0)
    x = x - np.mean(x, axis=0, keepdims=True)
    mx = np.max(np.abs(x)) + 1e-9
    x = x / mx
    y = np.tanh(1.2 * x)
    y = y * (peak / (np.max(np.abs(y)) + 1e-9))
    return y.squeeze()

# اگر رندر مستقیم pyfluidsynth مشکل داشت، مسیر fallback با midi2audio استفاده می‌شود.
def _render_wav_robust(pm, out_mid, out_wav, sf2_path=None, sr=22050):
    """همیشه اول MIDI را می‌نویسیم تا ارزیابی‌ها fail نشن."""
    sf2_path = sf2_path or SF2_PATH_DEFAULT
    pm.write(out_mid)
    # تلاش ۱: pyfluidsynth
    try:
        audio = pm.fluidsynth(fs=sr, sf2_path=sf2_path)
        audio = _normalize_limit(audio)
        rms = np.sqrt(np.mean(audio**2))
        if (rms < 1e-4) or (not np.isfinite(rms)):
            raise ValueError("silent_or_invalid_audio")
        sf.write(out_wav, audio, sr)
        return out_wav, sr
    except Exception as e:
        print("[render] fallback to midi2audio:", e)
        FluidSynth(sound_font=sf2_path, sample_rate=sr).midi_to_audio(out_mid, out_wav)
        y, sr2 = sf.read(out_wav, always_2d=False)
        y = _normalize_limit(y)
        sf.write(out_wav, y, sr2)
        return out_wav, sr2

def _to_mp3(wav_path, mp3_path, bitrate="192k"):
    try:
        AudioSegment.from_file(wav_path).export(mp3_path, format="mp3", bitrate=bitrate)
        return mp3_path
    except Exception as e:
        print("[mp3] export error:", e)
        return None

# ----------------------------- Emoji & Maps ---------------------------
EMOJI = {"ANGRY":"😠","FEAR":"😨","HAPPY":"😃","SAD":"😢","SURPRISE":"😲","OTHER":"😐"}

# ------------------------- Pipelines متن/صوت --------------------------

# پارس‌برت اختیاری—اگر نبود، انگلیسی کار می‌کند
try:
    pipe_fa = pipeline(
        "text-classification",
        model="/content/final_parsbert_emotion",
        tokenizer="/content/final_parsbert_emotion",
        device=DEVICE
    )
except Exception as e:
    pipe_fa = None
    print("[warn] Persian text model not found -> using English when needed")

pipe_en = pipeline(
    "text-classification",
    model="nateraw/bert-base-uncased-emotion",
    tokenizer="nateraw/bert-base-uncased-emotion",
    device=DEVICE,
    return_all_scores=False
)

def map_en(label):
    l = label.lower()
    if l in ["joy","love"]: return "HAPPY"
    if l == "sadness":      return "SAD"
    if l == "anger":        return "ANGRY"
    if l == "fear":         return "FEAR"
    if l == "surprise":     return "SURPRISE"
    return "OTHER"

audio_clf = pipeline("audio-classification", model="superb/wav2vec2-base-superb-er", device=DEVICE, top_k=None)
AUD_MAP = {"hap":"HAPPY","sad":"SAD","ang":"ANGRY","fea":"FEAR","sur":"SURPRISE","neu":"OTHER"}

# ------------------------- مدل تصویر (TF روی CPU) ---------------------
emotion_cnn = load_model("/content/emotion_model.h5", compile=False)
FER_LABELS = ['angry','disgust','fear','happy','sad','surprise','neutral']
IMG_MAP = {
    "angry":"ANGRY","disgust":"ANGRY","fear":"FEAR",
    "happy":"HAPPY","sad":"SAD","surprise":"SURPRISE","neutral":"OTHER"
}
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# -------------------------- تشخیص احساس‌ها ----------------------------
def analyze_text(text, threshold):
    if not text or not str(text).strip():
        return "OTHER", 0.0, EMOJI["OTHER"]
    try:
        lang = detect(text)
    except:
        lang = "fa"
    if lang == "fa" and pipe_fa is not None:
        out = pipe_fa(text)[0]
        idx = int(out["label"].split("_")[-1])
        FA = ['ANGRY','FEAR','HAPPY','HATE','OTHER','SAD','SURPRISE']
        label = FA[idx] if FA[idx]!="HATE" else "ANGRY"
        score = float(out["score"])
    else:
        out = pipe_en(text)[0]
        label = map_en(out["label"])
        score = float(out["score"])
    if score < threshold:
        label = "OTHER"
    return label, round(score,3), EMOJI[label]

# اگر تصویر چهره تشخیص داده نشد، وسط تصویر را به‌عنوان ROI می‌گیرد.
# تصویر را خاکستری می‌کند، چهره را با هار پیدا می‌کند؛ اگر نکند، مرکز تصویر را برش می‌زند.
def analyze_image(img, threshold):
    try:
        gray = cv2.cvtColor(np.array(img.convert("RGB")), cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        if len(faces)==0:
            h,w = gray.shape
            sz = min(h,w)//2
            x = (w-sz)//2; y = (h-sz)//2
            roi = gray[y:y+sz, x:x+sz]
        else:
            x,y,w,h = faces[0]
            roi = gray[y:y+h, x:x+w]
        face = cv2.resize(roi, (64,64), interpolation=cv2.INTER_AREA)
        face = face.astype("float32")/255.0
        face = face.reshape(1,64,64,1)
        preds = emotion_cnn.predict(face, verbose=0)[0]
        idx   = int(np.argmax(preds))
        score = float(preds[idx])
        raw   = FER_LABELS[idx]
        label = IMG_MAP.get(raw, "OTHER")
        return label, round(score,3), EMOJI[label]
    except Exception as e:
        print("[analyze_image error]", repr(e))
        return "OTHER", 0.0, EMOJI["OTHER"]

def analyze_audio(filepath, threshold):
    try:
        out = audio_clf(filepath)
        if not out:
            return "OTHER", 0.0, EMOJI["OTHER"]
        top = max(out, key=lambda x: x["score"])
        raw = top["label"].lower()
        score = float(top["score"])
        label = AUD_MAP.get(raw[:3], "OTHER")
        if score < threshold:
            label = "OTHER"
        return label, round(score,3), EMOJI[label]
    except Exception as e:
        print("[analyze_audio error]", repr(e))
        return "OTHER", 0.0, EMOJI["OTHER"]

def analyze_all(text, image, audio_path, threshold):
    if text and str(text).strip():
        return analyze_text(text, threshold)
    if image is not None:
        return analyze_image(image, threshold)
    if audio_path:
        return analyze_audio(audio_path, threshold)
    return "OTHER", 0.0, EMOJI["OTHER"]

# --------------------- تنظیمات ساخت موسیقی ----------------
GM = dict(
    piano=[0,1,4,5,6],
    guitar_ac=[24,25], guitar_el=[26,27,28],
    guitar_drive=[29,30],
    bass=[32,33,34,39],
    strings=[48,49,50],
    brass=[60,61],
    mallet=[11,12,13,14],
    pad=[88,89,91,94],
    lead=[80,81,82,84]
)

# سازهای ایرانی (تقریب با GM)
PERSIAN_GM = dict(
    santur=[15],               # Dulcimer ≈ Santur
    tar=[24,104,107],          # NylonGtr / Sitar / Koto
    setar=[24,104],
    oud=[24,25,104],
    kamancheh=[40],            # Violin
    ney=[77,75],               # Shakuhachi / Pan Flute
    qanun=[46,15,107],         # Harp / Dulcimer / Koto
)

# تنها سازهای ایرانی در حالت Persian
PERSIAN_ROLES = {
    "HAPPY":    ["santur","qanun","oud","ney","kamancheh","tar","setar"],
    "SAD":      ["kamancheh","ney","santur","qanun","setar","tar"],
    "ANGRY":    ["santur","tar","kamancheh","ney","oud"],   # خشم ایرانی بدون گیتار غربی
    "FEAR":     ["ney","kamancheh","santur","qanun"],
    "SURPRISE": ["santur","qanun","oud","ney","kamancheh"],
    "OTHER":    ["santur","qanun","ney","kamancheh","setar"]
}

EMO_PROFILE = {
    "HAPPY":    {"mode":"major","tempo_range":(124,142),"base_keys":[60,62,65,67],
                 "progressions":[["I","V","vi","IV"],["I","IV","V","I"],["I","V","IV","I"]],
                 "melody_density_range":(0.7,0.95),"drums":True,"comment":"شاد و رقصان"},
    "SAD":      {"mode":"minor","tempo_range":(45,60),"base_keys":[57,55,52],
                 "progressions":[["i","VI","III","VII"],["i","iv","i","VII"],["i","VI","i","v"]],
                 "melody_density_range":(0.12,0.28),"drums":False,"comment":"خیلی آرام و افسرده"},
    "ANGRY":    {"mode":"minor","tempo_range":(150,175),"base_keys":[50,48,53],
                 "progressions":[["i","bVI","bVII","i"],["i","bII°","bVII","i"]],
                 "melody_density_range":(0.8,0.96),"drums":True,"comment":"راک/متال تهاجمی"},
    "FEAR":     {"mode":"minor","tempo_range":(50,68),"base_keys":[57,58,55],
                 "progressions":[["i","bII°","i","bVI"],["i","iv","bII°","i"]],
                 "melody_density_range":(0.12,0.3),"drums":False,"comment":"دلهره‌آور"},
    "SURPRISE": {"mode":"major","tempo_range":(122,145),"base_keys":[65,67,69],
                 "progressions":[["I","V","IV","♭VII"],["I","IV","V","♭VII"]],
                 "melody_density_range":(0.6,0.95),"drums":True,"comment":"جهشی/سینکوپ"},
    "OTHER":    {"mode":"major","tempo_range":(90,110),"base_keys":[60,62],
                 "progressions":[["I","ii","IV","V"],["I","IV","ii","V"]],
                 "melody_density_range":(0.4,0.65),"drums":False,"comment":"خنثی/پس‌زمینه"},
}

MAJOR_ROMAN = {"I":(0,"maj"),"ii":(2,"min"),"iii":(4,"min"),"IV":(5,"maj"),"V":(7,"maj"),"vi":(9,"min"),"♭VII":(10,"maj")}
MINOR_ROMAN = {"i":(0,"min"),"ii°":(2,"dim"),"III":(3,"maj"),"iv":(5,"min"),"v":(7,"min"),
               "VI":(8,"maj"),"VII":(10,"maj"),"bVI":(8,"maj"),"bVII":(10,"maj"),"bII°":(1,"dim")}
QUAL2INTS = {"maj":[0,4,7], "min":[0,3,7], "dim":[0,3,6]}

def _scale_for(mode:str, emotion:str, style:str):
    if style != "persian":
        return [0,2,4,5,7,9,11] if mode=="major" else [0,2,3,5,7,8,10]
    # تقریبی از دستگاه‌ها (بدون ربع‌پرده)
    if emotion in ["HAPPY","SURPRISE"]: return [0,1,4,5,7,8,11]   # Hijaz / Phrygian-dominant
    if emotion in ["SAD"]:              return [0,1,3,5,7,8,10]   # Phrygian (حزن)
    if emotion in ["FEAR"]:             return [0,1,3,5,6,8,10]   # Saba-like (دلهره)
    return [0,2,3,5,7,8,10]             # Nahawand-like

def _add_chord(inst, start, end, root, qual, rng, spread=0):
    for i in QUAL2INTS[qual]:
        p = int(root+i + spread)
        vel = int(np.clip(72 + rng.randint(-8,8), 40, 105))
        inst.notes.append(pretty_midi.Note(velocity=vel, pitch=p, start=start, end=end))

def _add_persian_bed(inst, start, end, root, rng, add_fifth=True, octave_spread=True):
    base = int(root)
    p_list = [base]
    if add_fifth: p_list.append(base+7)
    if octave_spread and (rng.random()<0.6):
        p_list += [base-12, base+12]
    vel = int(np.clip(78 + rng.randint(-6,6), 50, 105))
    for p in p_list:
        p = max(0, min(127, p))
        inst.notes.append(pretty_midi.Note(velocity=vel, pitch=p, start=start, end=end))

def _arp_pattern(inst, start, end, root, qual, rng, step):
    ints = QUAL2INTS[qual]; seq = ints + ints[::-1]
    t = start; idx = rng.randint(0,len(seq)-1)
    while t < end:
        p = int(root + seq[idx % len(seq)])
        dur = step * (1.0 if rng.random()<0.7 else 0.5)
        vel = int(np.clip(74 + rng.randint(-12,12), 40, 110))
        jit = rng.uniform(-0.015,0.015)
        inst.notes.append(pretty_midi.Note(velocity=vel, pitch=p, start=t+jit, end=min(t+jit+dur, end)))
        t += step; idx += 1

def _scale_run(inst, start, end, key, scale, rng, step):
    t = start; idx = rng.randint(0,len(scale)-1); direction = rng.choice([1,-1])
    last = key + scale[idx]
    while t < end:
        deg = scale[idx % len(scale)]
        base = key + deg + rng.choice([-12,0,12])
        pitch = int(np.clip(int(0.65*last + 0.35*base), key-7, key+19))
        dur = step if rng.random()<0.8 else step*2
        vel = int(np.clip(76 + rng.randint(-10,10), 45, 110))
        jit = rng.uniform(-0.015,0.015)
        inst.notes.append(pretty_midi.Note(velocity=vel, pitch=pitch, start=t+jit, end=min(t+jit+dur, end)))
        last = pitch; t += step; idx += direction
        if rng.random()<0.12: direction *= -1

def _bass_pattern(inst, start, end, root, bpm, rng, style="eighths"):
    beat = 60.0/bpm
    if style=="long":
        p = max(0, root-12)
        inst.notes.append(pretty_midi.Note(velocity=74, pitch=p, start=start, end=end))
    elif style=="syncop":
        for k in [0.5, 1.5, 2.5, 3.5]:
            t = start + k*beat + rng.uniform(-0.01,0.01)
            inst.notes.append(pretty_midi.Note(velocity=85+rng.randint(-6,6), pitch=max(0,root-12), start=t, end=min(t+0.30, end)))
    else:
        t = start
        while t < end:
            p = max(0, root-12)
            d = 0.45*beat
            inst.notes.append(pretty_midi.Note(velocity=82+rng.randint(-8,8), pitch=p, start=t, end=min(t+d, end)))
            t += 0.5*beat

def _lead_melody(inst, start, bars, bpm, mode, key, density, rng, scale_override=None):
    scale = scale_override if scale_override is not None else ([0,2,4,5,7,9,11] if mode=="major" else [0,2,3,5,7,8,10])
    step = 0.5*(60.0/bpm)       # 8th
    total_sub = int(bars*4*2)
    t = start; last_pitch = key + 7 + rng.choice([-12,0,12])
    lo, hi = key-7, key+19
    motif = [rng.choice(scale) for _ in range(rng.choice([3,4,5,6]))]
    for s in range(total_sub):
        if rng.random() < density:
            deg = motif[s % len(motif)] if rng.random()<0.75 else rng.choice(scale)
            base = key + deg + rng.choice([-12,0,12])
            pitch = int(np.clip(int(0.65*last_pitch + 0.35*base), lo, hi))
            dur = step if rng.random()<0.65 else step*2
            jit = rng.uniform(-0.02,0.02)
            vel = int(np.clip(80 + rng.randint(-14,14), 40, 115))
            inst.notes.append(pretty_midi.Note(velocity=vel, pitch=pitch, start=t+jit, end=t+jit+dur))
            last_pitch = pitch
        t += step

# --- درام‌ها
def _drum_bar(inst_drm, bar_start, beat_dur, rng, style="straight"):
    """درام ساده‌ی 4/4"""
    for b in range(4):
        t = bar_start + b*beat_dur
        inst_drm.notes.append(pretty_midi.Note(100 if b%2==0 else 92, pitch=36 if b%2==0 else 38, start=t, end=t+0.06))
        inst_drm.notes.append(pretty_midi.Note(62, pitch=42, start=t, end=t+0.03))
        if style=="swing":
            inst_drm.notes.append(pretty_midi.Note(62, pitch=42, start=t+beat_dur*2/3, end=t+beat_dur*2/3+0.03))
        else:
            inst_drm.notes.append(pretty_midi.Note(62, pitch=42, start=t+beat_dur/2, end=t+beat_dur/2+0.03))

def _dance_4onfloor(inst_drm, bar_start, beat_dur, rng):
    """چهار-به-کف برای شاد غربی"""
    for b in range(4):
        t = bar_start + b*beat_dur
        # Kick on every beat
        inst_drm.notes.append(pretty_midi.Note(108, pitch=36, start=t, end=t+0.08))
        # Snare/Clap on 2 & 4
        if b in [1,3]:
            inst_drm.notes.append(pretty_midi.Note(110, pitch=38, start=t, end=t+0.06))
            inst_drm.notes.append(pretty_midi.Note(120, pitch=39, start=t, end=t+0.03))  # clap
        # Hi-hat 8ths
        inst_drm.notes.append(pretty_midi.Note(70, pitch=42, start=t, end=t+0.03))
        inst_drm.notes.append(pretty_midi.Note(70, pitch=42, start=t+beat_dur/2, end=t+beat_dur/2+0.03))

def _metal_bar(inst_drm, bar_start, beat_dur, rng):
    """درام متال: دابل‌کیک + کرش آغاز هر میزان"""
    # Crash on bar start
    inst_drm.notes.append(pretty_midi.Note(100, pitch=49, start=bar_start, end=bar_start+0.5*beat_dur))
    # 16th grid
    for s in range(16):
        t = bar_start + s*(beat_dur/4)
        # Snare on beats 2 and 4
        if abs(t - (bar_start+beat_dur)) < 1e-6 or abs(t - (bar_start+3*beat_dur)) < 1e-6:
            inst_drm.notes.append(pretty_midi.Note(110, pitch=38, start=t, end=t+0.05))
        else:
            # Double kick dense (avoid exactly at snare)
            inst_drm.notes.append(pretty_midi.Note(100, pitch=36, start=t, end=t+0.04))
        # Hi-hat 8ths
        if s % 2 == 0:
            inst_drm.notes.append(pretty_midi.Note(72, pitch=42, start=t, end=t+0.03))

def _persian_68(inst_drm, bar_start, beat_dur, rng):
    """ریتم شش‌و‌هشت ایرانی شبه‌تنبک/دف با سازهای GM (کونگا/تمبورین/کِلَپ)"""
    step = beat_dur*(2/3)  # 6 پالس در هر میزان 4/4 → حس 6/8
    pulses = [bar_start + i*step for i in range(6)]
    # Dum روی 1 و 4 (کونگای بم)
    for i in [0,3]:
        inst_drm.notes.append(pretty_midi.Note(105, pitch=64, start=pulses[i], end=pulses[i]+0.08))  # Low Conga
    # Tek روی 3 و 6 (کونگای زیر/باز)
    for i in [2,5]:
        inst_drm.notes.append(pretty_midi.Note(96, pitch=63, start=pulses[i], end=pulses[i]+0.06))  # Open High Conga
    # تمبورین ریز
    for i in range(6):
        inst_drm.notes.append(pretty_midi.Note(70, pitch=54, start=pulses[i], end=pulses[i]+0.03))
    # دست‌زدن روی 4
    inst_drm.notes.append(pretty_midi.Note(120, pitch=39, start=pulses[3], end=pulses[3]+0.04))

def _drum_fill(inst_drm, bar_start, beat_dur, rng):
    for i, pitch in enumerate([47,45,43,41]):
        t = bar_start + i*(beat_dur/2)
        inst_drm.notes.append(pretty_midi.Note(86+rng.randint(-6,6), pitch=pitch, start=t, end=t+0.12))

# MIDI Control Change (ولوم، پنینگ و…)
def _cc(inst, cc, value, time=0.0):
    inst.control_changes.append(pretty_midi.ControlChange(cc, int(value), time))

def _power_chord_riff(inst_L, inst_R, start, end, root, bpm, rng):
    beat = 60.0/bpm; t = start
    while t < end:
        dur = min(0.45*beat, end-t)
        velL = int(np.clip(110 + rng.randint(-8,8), 60, 127))
        velR = int(np.clip(108 + rng.randint(-8,8), 60, 127))
        for off in [0,7,12]:
            inst_L.notes.append(pretty_midi.Note(velocity=velL, pitch=int(root+off), start=t+rng.uniform(-0.005,0.005), end=min(t+dur, end)))
            inst_R.notes.append(pretty_midi.Note(velocity=velR, pitch=int(root+off), start=t+rng.uniform(-0.005,0.005), end=min(t+dur, end)))
        t += 0.5*beat  # 8th push

def generate_full_track(emotion: str, minutes: float = 3.0, seed: int | None = None, style: str = "global", sf2_path: str | None = None):
    assert emotion in EMO_PROFILE, f"unknown emotion {emotion}"
    rng = random.Random(seed if seed is not None else int(time.time()*1000) % 2_147_483_647)
    prof = EMO_PROFILE[emotion]
    bpm = rng.randint(*prof["tempo_range"])
    key = rng.choice(prof["base_keys"])
    progression = rng.choice(prof["progressions"])
    density = rng.uniform(*prof["melody_density_range"])
    is_persian = (style == "persian")
    sec_per_bar = 4*(60.0/bpm)
    total_bars = max(16, int((minutes*60)/sec_per_bar))
    bars_A = int(total_bars*0.4); bars_B = int(total_bars*0.4); bars_A2 = total_bars - (bars_A+bars_B)
    beat_dur = (60.0/bpm)
    mode_here = prof["mode"]
    ROM = MAJOR_ROMAN if mode_here=="major" else MINOR_ROMAN
    scale_local = _scale_for(mode_here, emotion, style)

    pm = pretty_midi.PrettyMIDI()
    insts = {}

    if is_persian:
        roles_src = PERSIAN_ROLES[emotion][:]
        rng.shuffle(roles_src)
        roles = roles_src[:rng.randint(3, min(6, len(roles_src)))]
        bed_prog = rng.choice(PERSIAN_GM["kamancheh"] + PERSIAN_GM["santur"])
        inst_har = pretty_midi.Instrument(program=bed_prog); pm.instruments.append(inst_har)
        _cc(inst_har,7,96); _cc(inst_har,10,64)

        def _addp(name, progs, vol, pan):
            insts[name] = pretty_midi.Instrument(program=rng.choice(progs))
            _cc(insts[name],7,vol); _cc(insts[name],10,pan)
            pm.instruments.append(insts[name])

        if "santur"    in roles: _addp("santur", PERSIAN_GM["santur"],   100, 60)
        if "qanun"     in roles: _addp("qanun",  PERSIAN_GM["qanun"],     98, 68)
        if "tar"       in roles: _addp("tar",    PERSIAN_GM["tar"],       96, 56)
        if "setar"     in roles: _addp("setar",  PERSIAN_GM["setar"],     94, 72)
        if "oud"       in roles: _addp("oud",    PERSIAN_GM["oud"],       98, 52)
        if "kamancheh" in roles: _addp("kam",    PERSIAN_GM["kamancheh"], 96, 76)
        if "ney"       in roles: _addp("ney",    PERSIAN_GM["ney"],      100, 64)

        # فقط برای HAPPY ایرانی: ریتم 6/8 (daf/tonbak شبیه‌سازی با درام GM)
        inst_drm = pretty_midi.Instrument(program=0, is_drum=True) if emotion=="HAPPY" else None
        if inst_drm: pm.instruments.append(inst_drm)
    else:
        roles_pool = ["pad","piano","guitar_ac","guitar_el","bass","strings","lead","mallet","brass","guitar_drive"]
        rng.shuffle(roles_pool)
        roles = roles_pool[:rng.randint(3, 6)]

        # اجباری‌ها برای انرژی بهتر
        if emotion=="ANGRY":
            if "guitar_drive" not in roles: roles.append("guitar_drive")
            if "bass" not in roles: roles.append("bass")
            prof["drums"] = True
        if emotion=="HAPPY":
            if "bass" not in roles: roles.append("bass")
            if "brass" not in roles: roles.append("brass")
            prof["drums"] = True

        # Harmony bed
        if "pad" in roles:
            inst_har = pretty_midi.Instrument(program=rng.choice(GM["pad"]+GM["strings"]))
        elif "strings" in roles:
            inst_har = pretty_midi.Instrument(program=rng.choice(GM["strings"]))
        else:
            inst_har = pretty_midi.Instrument(program=rng.choice(GM["piano"]))
        _cc(inst_har, 7, 92); _cc(inst_har, 10, 64)
        pm.instruments.append(inst_har)

        def _addi(name, progs, vol, pan):
            if name in insts: return
            insts[name] = pretty_midi.Instrument(program=rng.choice(progs))
            _cc(insts[name],7,vol); _cc(insts[name],10,pan)
            pm.instruments.append(insts[name])

        if "piano" in roles:        _addi("piano", GM["piano"], 96, 64)
        if "guitar_ac" in roles:    _addi("gtr_ac", GM["guitar_ac"], 96, 52)
        if "guitar_el" in roles:    _addi("gtr_el", GM["guitar_el"], 96, 76)
        if "guitar_drive" in roles: _addi("gtrL", GM["guitar_drive"], 110, 32); _addi("gtrR", GM["guitar_drive"], 110, 96)
        if "bass" in roles:         _addi("bass", GM["bass"], 110, 64)
        if "strings" in roles:      _addi("str2", GM["strings"], 90, 84)
        if "brass" in roles:        _addi("brs", GM["brass"], 96, 70)
        if "mallet" in roles:       _addi("mlt", GM["mallet"], 92, 58)
        if "lead" in roles:         _addi("lead", GM["lead"], 104, 64)
        inst_drm = pretty_midi.Instrument(program=0, is_drum=True) if prof["drums"] else None
        if inst_drm: pm.instruments.append(inst_drm)

    def render_section(start_bar, num_bars, key_this, prog_this, dens_mul=1.0, final_fill=False):
        for i in range(num_bars):
            roman = prog_this[i % len(prog_this)]
            #  فالسِیف در صورت درجه ناشناخته
            if roman not in ROM:
                off, qual = (0, "maj" if mode_here=="major" else "min")
            else:
                off, qual = ROM[roman]
            root = (key_this + off) % 128
            start = (start_bar+i)*4*beat_dur
            end   = (start_bar+i+1)*4*beat_dur

            if is_persian:
                _add_persian_bed(inst_har, start, end, root, rng, add_fifth=True, octave_spread=True)
                step = 0.5*beat_dur
                for name in ["santur","qanun","tar","setar","oud"]:
                    if name in insts and rng.random()<0.85:
                        _scale_run(insts[name], start, end, key_this, scale_local, rng, step=step)
                if "ney" in insts and rng.random()<0.95:
                    _scale_run(insts["ney"], start, end, key_this, scale_local, rng, step=step)
                if inst_drm:
                    _persian_68(inst_drm, start, beat_dur, rng)
            else:
                _add_chord(inst_har, start, end, root, qual, rng, spread=rng.choice([0,12,-12,0]))
                has_drive = ("gtrL" in insts and "gtrR" in insts)
                if has_drive and emotion=="ANGRY":
                    _power_chord_riff(insts["gtrL"], insts["gtrR"], start, end, root, bpm, rng)
                else:
                    gtr = insts.get("gtr_el") or insts.get("gtr_ac")
                    if gtr: _arp_pattern(gtr, start, end, root, qual, rng, step=0.5*beat_dur)
                if "piano" in insts and (emotion!="ANGRY") and rng.random()<0.85:
                    _arp_pattern(insts["piano"], start, end, root, qual, rng, step=0.5*beat_dur)
                if "brs" in insts and rng.random()<0.4 and emotion in ["HAPPY","SURPRISE"]:
                    _add_chord(insts["brs"], start, min(start+beat_dur, end), root+12, qual, rng)
                if "mlt" in insts and rng.random()<0.5:
                    _arp_pattern(insts["mlt"], start, min(start+2*beat_dur, end), root+12, qual, rng, step=0.25*beat_dur)
                if "bass" in insts:
                    style_b = "eighths"
                    if emotion in ["SAD","FEAR"]: style_b = "long"
                    if emotion in ["HAPPY","SURPRISE"]: style_b = rng.choice(["eighths","syncop"])
                    if emotion=="ANGRY": style_b = "eighths"
                    _bass_pattern(insts["bass"], start, end, root, bpm, rng, style=style_b)
                if inst_drm:
                    if emotion=="HAPPY":      _dance_4onfloor(inst_drm, start, beat_dur, rng)
                    elif emotion=="ANGRY":    _metal_bar(inst_drm, start, beat_dur, rng)
                    elif emotion=="SURPRISE": _drum_bar(inst_drm, start, beat_dur, rng, style="swing" if rng.random()<0.4 else "straight")
                    elif emotion not in ["SAD","FEAR"]:
                        _drum_bar(inst_drm, start, beat_dur, rng, style="straight")

            # FEAR (Global): ارگ کلیسا لانگ‌نوت
            if (not is_persian) and emotion=="FEAR":
                org = pretty_midi.Instrument(program=19)  # Church Organ
                _cc(org,7,98); _cc(org,10,64)
                org.notes.append(pretty_midi.Note(velocity=84, pitch=int(root), start=start, end=end))
                pm.instruments.append(org)

        # Lead
        if is_persian:
            lead_inst = insts.get("ney") or insts.get("kam") or inst_har
            dens = density*dens_mul*(0.5 if emotion=="SAD" else 1.0)
            _lead_melody(lead_inst, start_bar*4*beat_dur, num_bars, bpm, mode_here, key_this, dens, rng, scale_override=scale_local)
        else:
            lead_inst = insts.get("lead") or insts.get("piano") or inst_har
            dmul = density*dens_mul
            if emotion=="SAD": dmul *= 0.6
            _lead_melody(lead_inst, start_bar*4*beat_dur, num_bars, bpm, mode_here, key_this, dmul, rng)
        if final_fill and (not is_persian) and inst_drm and (emotion not in ["SAD","FEAR"]):
            _drum_fill(inst_drm, (start_bar+num_bars-1)*4*beat_dur, beat_dur, rng)


# بخش A با پروگرشن اصلی،
# بخش B با مدولاسیون کلید و پروگرشن جدید،
# بخش A′ برگشت به کلید و پروگرشن اول (با دانسیته کمی متفاوت).
    # ساختار A–B–A′
    render_section(0, bars_A, key, progression, dens_mul=1.0, final_fill=True)
    mod_key = key + rng.choice([-2, 2, 0])
    new_prog = rng.choice(prof["progressions"])
    render_section(bars_A, bars_B, mod_key, new_prog, dens_mul=rng.uniform(0.9,1.25), final_fill=True)
    render_section(bars_A+bars_B, bars_A2, key, progression, dens_mul=rng.uniform(0.85,1.15), final_fill=False)

    stamp = int(time.time())
    out_mid = f"{emotion.lower()}_{style}_{minutes:.1f}min_{stamp}.mid"
    out_wav = f"{emotion.lower()}_{style}_{minutes:.1f}min_{stamp}.wav"
    wav_path, sr = _render_wav_robust(pm, out_mid, out_wav, sf2_path=sf2_path, sr=22050)

    # MP3
    out_mp3 = f"{emotion.lower()}_{style}_{minutes:.1f}min_{stamp}.mp3"
    _ = _to_mp3(wav_path, out_mp3, bitrate="192k")

    meta = {
        "emotion": emotion, "style": style, "minutes": minutes, "bpm": bpm, "mode": mode_here,
        "key_midi": key, "progression_A": progression, "progression_B": new_prog,
        "drums": (not is_persian) and bool(EMO_PROFILE[emotion]["drums"]) or (is_persian and emotion=="HAPPY"),
        "comment": prof["comment"], "bars_total": total_bars, "sr": sr
    }
    return out_mid, wav_path, out_mp3, meta

# ---------------------------- ارزیابی موسیقی --------------------------

# اهداف ارزیابی برای هر احساس: مُد، بازه تمپو، بازه دانسیته نت/میـزان، علاقه به دیسونانس/سینکوپ.

TARGETS = {
    "HAPPY":    dict(mode="major", tempo=(124,142), density=(7,12), want_dissonance=False, want_syncopation=True),
    "SAD":      dict(mode="minor", tempo=(45,60),   density=(1,4),  want_dissonance=False, want_syncopation=False),
    "ANGRY":    dict(mode="minor", tempo=(150,175), density=(8,14), want_dissonance=True,  want_syncopation=True),
    "FEAR":     dict(mode="minor", tempo=(50,68),   density=(1,4),  want_dissonance=True,  want_syncopation=False),
    "SURPRISE": dict(mode="major", tempo=(122,145), density=(6,12), want_dissonance=False, want_syncopation=True),
    "OTHER":    dict(mode="major", tempo=(85,110),  density=(4,8),  want_dissonance=False, want_syncopation=False),
}

def analyze_midi_struct(mid_path, emotion, build_params):
    pm = pretty_midi.PrettyMIDI(mid_path)
    non_drums = [ins for ins in pm.instruments if not ins.is_drum]
    note_count = sum(len(ins.notes) for ins in non_drums)
    bars = build_params["bars_total"]
    density = note_count / max(1,bars)
    bpm_guess = build_params["bpm"]

    step = 60.0/bpm_guess/2
    sync_onsets, total_notes = 0, 0
    for ins in non_drums:
        for n in ins.notes:
            total_notes += 1
            frac = (n.start/step) % 2
            if 0.4 < frac < 1.6: sync_onsets += 1
    sync_ratio = (sync_onsets / max(1,total_notes)) if total_notes else 0.0
    has_dim_like = any((n.end-n.start) < (60.0/bpm_guess)/4 for ins in non_drums for n in ins.notes)

    target = TARGETS[emotion]
    mode_match = 1.0 if (build_params["mode"] == target["mode"]) else 0.0
    tempo_fit  = 1.0 if (target["tempo"][0] <= bpm_guess <= target["tempo"][1]) else 0.0
    d_lo, d_hi = target["density"]
    if density<d_lo: density_fit = max(0.0, 1 - (d_lo-density)/max(1e-6,d_lo))
    elif density>d_hi: density_fit = max(0.0, 1 - (density-d_hi)/max(1e-6,d_hi))
    else: density_fit = 1.0
    sync_fit   = min(1.0, sync_ratio*1.5) if target["want_syncopation"] else (1.0 - min(1.0, sync_ratio*1.5))
    disson_fit = 1.0 if (target["want_dissonance"] and has_dim_like) or (not target["want_dissonance"] and not has_dim_like) else 0.5

    score = 0.25*mode_match + 0.25*tempo_fit + 0.25*density_fit + 0.15*disson_fit + 0.10*sync_fit
    return {
        "bpm": bpm_guess, "bars": bars, "note_density_per_bar": round(density,2),
        "mode_match": round(mode_match,2), "tempo_fit": round(tempo_fit,2),
        "density_fit": round(density_fit,2), "dissonance_fit": round(disson_fit,2),
        "syncopation_fit": round(sync_fit,2), "overall_structural_score": round(float(score),3),
        "build_params": build_params,
    }

def evaluate_with_audio_classifier(wav_path, target_emotion:str):
    try:
        out = audio_clf(wav_path)
        top = max(out, key=lambda x:x["score"])
        mapped = AUD_MAP.get(top["label"].lower()[:3], "OTHER")
        return {"predicted_emotion_audio": mapped,
                "predicted_confidence_audio": round(float(top["score"]),3),
                "target_emotion": target_emotion,
                "audio_match": 1.0 if mapped==target_emotion else 0.0}
    except Exception as e:
        print("[audio_eval error]", repr(e))
        return {"predicted_emotion_audio": None,"predicted_confidence_audio":0.0,
                "target_emotion": target_emotion,"audio_match":0.0}

def generate_and_evaluate_full(emotion:str, minutes:float=3.0, seed=None, style:str="global", sf2_path: str | None = None):
    mid_path, wav_path, mp3_path, meta = generate_full_track(emotion, minutes=minutes, seed=seed, style=style, sf2_path=sf2_path)
    struct = analyze_midi_struct(mid_path, emotion, meta)
    audio_eval = evaluate_with_audio_classifier(wav_path, emotion)
    final_score = round(0.75*struct["overall_structural_score"] + 0.25*audio_eval["audio_match"], 3)
    report = {"emotion": emotion, "style": style, "structural": struct, "audio_eval": audio_eval, "final_score": final_score}
    return wav_path, mp3_path, report

# ----------------------------- Gradio UI ------------------------------
def analyze_and_make_music(text, image, audio_path, threshold, minutes, seed, style_choice, sf2_upload):
    label, score, emoji = analyze_all(text, image, audio_path, threshold)
    try:
        seed_int = None if (seed is None or str(seed).strip()=="") else int(seed)
    except:
        seed_int = None
    style = "persian" if (str(style_choice).lower().startswith("pers")) else "global"
    sf2_path = str(sf2_upload) if (sf2_upload is not None and str(sf2_upload).strip()!="") else None

    # تولید + ارزیابی
    wav_path, mp3_path, rep = generate_and_evaluate_full(label, minutes=float(minutes), seed=seed_int, style=style, sf2_path=sf2_path)

    # تبدیل برای پخش آنلاین
    y, sr = sf.read(wav_path, always_2d=False)
    if hasattr(y, "ndim") and y.ndim > 1:
        y = np.mean(y, axis=1)
    y = _normalize_limit(y)

    rep["detected_label"] = label
    rep["detected_confidence"] = score
    rep["soundfont"] = sf2_path if sf2_path else SF2_PATH_DEFAULT
    return label, score, emoji, (sr, y.astype(np.float32)), rep, (mp3_path if mp3_path else wav_path)

with gr.Blocks(css="""
  #root {max-width:980px;margin:auto;padding:16px;}
  .out {font-size:1.1rem;text-align:center;}
""") as app:
    gr.Markdown("## تشخیص احساس (متن/تصویر/صوت) + ساخت موسیقی ( خارجی/داخلی ) + گزارش ارزیابی + خروجی MP3")
    with gr.Row():
        with gr.Column():
            txt = gr.Textbox(label="متن (فارسی/انگلیسی)", lines=3, placeholder="مثال: امروز خیلی خوشحالم")
            img = gr.Image(label="تصویر چهره", type="pil")
            aud = gr.Audio(label="صوت (wav/mp3)", type="filepath")
        with gr.Column():
            thr  = gr.Slider(0,1,0.7,step=0.01, label="آستانه OTHER")
            mins = gr.Slider(1.0, 6.0, value=3.0, step=0.5, label="مدت موسیقی (دقیقه)")
            seed = gr.Textbox(value="", label="Seed (خالی = هر بار متفاوت)")
            style_dd = gr.Dropdown(choices=["Global/Western","Persian/Traditional"], value="Global/Western", label="سبک/Palette")
            sf2_file = gr.File(label="ساوند فونت دلخواه خودتو آپلود کن:", file_types=[".sf2"], type="filepath")
            go   = gr.Button("🔎 تحلیل و ساخت موسیقی", variant="primary")

    with gr.Row():
        o1 = gr.Textbox(label="احساس",    interactive=False, elem_classes="out")
        o2 = gr.Textbox(label="اعتماد",   interactive=False, elem_classes="out")
        o3 = gr.Textbox(label="ایموجی",   interactive=False, elem_classes="out")

    audio_out = gr.Audio(label="🎧 موسیقی خروجی (پخش آنلاین)", type="numpy")
    rep_out   = gr.JSON(label="📊 گزارش ارزیابی (ساختاری + کلاسیفایر صوت روی WAV)")
    mp3_out   = gr.File(label="⬇️ دانلود MP3")

    go.click(analyze_and_make_music,
             [txt, img, aud, thr, mins, seed, style_dd, sf2_file],
             [o1, o2, o3, audio_out, rep_out, mp3_out])

app.launch(share=False)
# =====================================================================

# app.launch(share=True)
# app.launch(debug=True)

### 2- معیارها | ارزیابی موسیقی

ساخت متاداده برای موسیقی های تولید شده و توصیفشان

In [ ]:
import os, pandas as pd, glob

GEN_DIR = "/content"
files = sorted([p for p in glob.glob(os.path.join(GEN_DIR, "*.wav"))])  # فقط WAV (تا با دیگر فرمت ها دوبل نشود)

rows = []
for p in files:
    name = os.path.basename(p).lower()
    if "happy" in name:  txt = "very upbeat dance pop / 6-8 energetic"
    elif "angry" in name: txt = "heavy metal angry: distorted electric guitars, loud drums, dark bass"
    elif "sad" in name:   txt = "very slow, mournful, crying, minimal, sparse"
    elif "fear" in name:  txt = "ominous church organ, eerie textures, suspense"
    elif "surprise" in name: txt = "syncopated bright brass, sudden accents"
    else: txt = "neutral background instrumental"
    rows.append({"file": p, "text": txt})

meta = pd.DataFrame(rows)
meta.to_csv("/content/meta.csv", index=False)


ارزیابی موسیقی تولیدی براساس متریک های موسیقی

In [ ]:
# ===================== Audio Eval (robust v4): CLAP, R-Prec, FAD, KAD, Diversity, MOS =====================
!pip -q install laion-clap torchvggish librosa pandas scipy tqdm soundfile tabulate matplotlib

import os, glob, json, math, zipfile, warnings
from typing import List
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import librosa
import torch
from tqdm import tqdm
from scipy import linalg

# ----------------------------- تنظیمات -----------------------------
GEN_DIR  = "/content"                 # فقط فایل‌های تولیدی‌ مورد ارزیابی
REF_DIR  = "/content/Ref_Eval_DIR"    # پوشه رفرنس‌ها؛ اگر خالی بود و FORCE_SELF_REF=True → از خود GEN استفاده می‌شود
META_CSV = "/content/meta.csv"        #  ستون‌های file,text
OUT_DIR  = "/content/eval_out"

FORCE_SELF_REF = True                 # اگر مرجع نداری، خود GEN را مرجع بگیر (FAD≈0 ولی null نمی‌شود)
AUTO_FAKE_MOS  = True                 # در نبود MOS واقعی، امتیازهای تصادفی بساز
N_FAKE_RATERS  = 15                   # تعداد رأی‌دهنده‌ی تصادفی برای MOS فیک

os.makedirs(OUT_DIR, exist_ok=True)

# ----------------------------- ابزارها -----------------------------
def list_audio_files(d, exts=(".wav",".mp3",".flac",".ogg",".m4a")):
    if not d or not os.path.isdir(d): return []
    fs = []
    for e in exts: fs += glob.glob(os.path.join(d, f"*{e}"))
    return sorted(fs)

def load_mono(path, sr=48000):
    y, _ = librosa.load(path, sr=sr, mono=True)
    y = np.nan_to_num(y, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)
    m = np.max(np.abs(y)) if y.size else 0.0
    if m > 1e-9: y = y / m
    return y

def cosine_sim(a, b, eps=1e-8):
    a = a / (np.linalg.norm(a, axis=-1, keepdims=True) + eps)
    b = b / (np.linalg.norm(b, axis=-1, keepdims=True) + eps)
    return (a * b).sum(-1)

def _robust_cov(X):
    """کوواریانس پایدار برای جلوگیری از NaN/Inf."""
    C = np.cov(X, rowvar=False)
    if not np.isfinite(C).all():
        C = np.cov(X, rowvar=False, bias=True)
    if not np.isfinite(C).all():
        X2 = X + 1e-6*np.random.randn(*X.shape)
        C = np.cov(X2, rowvar=False, bias=True)
    C = np.nan_to_num(C, nan=0.0, posinf=0.0, neginf=0.0)
    return (C + C.T) / 2.0

def frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    diff = mu1 - mu2
    I = eps*np.eye(sigma1.shape[0])
    covmean, _ = linalg.sqrtm((sigma1 + I) @ (sigma2 + I), disp=False)
    if not np.isfinite(covmean).all():
        covmean = linalg.sqrtm((sigma1 + 10*I) @ (sigma2 + 10*I))
    if np.iscomplexobj(covmean): covmean = covmean.real
    val = diff.dot(diff) + np.trace(sigma1 + sigma2 - 2*covmean)
    if not np.isfinite(val): val = float(np.linalg.norm(diff))
    return float(val)

# ----------------------------- CLAP ---------------------------------
import laion_clap
clap_device = "cuda" if torch.cuda.is_available() else "cpu"
clap_model = laion_clap.CLAP_Module(enable_fusion=False, device=clap_device)
clap_model.load_ckpt()

@torch.no_grad()
def embed_audio_clap(paths: List[str], sr=48000, batch_size=8) -> np.ndarray:
    outs = []
    for i in tqdm(range(0, len(paths), batch_size), desc="CLAP audio embed"):
        batch = paths[i:i+batch_size]
        wavs = [torch.tensor(load_mono(p, sr), dtype=torch.float32) for p in batch]
        e = clap_model.get_audio_embedding_from_data(x=wavs, use_tensor=True)
        outs.append(e.cpu().numpy())
    return np.concatenate(outs, axis=0) if outs else np.zeros((0,512), dtype=np.float32)

@torch.no_grad()
def embed_text_clap(texts: List[str], batch_size=16) -> np.ndarray:
    outs = []
    for i in tqdm(range(0, len(texts), batch_size), desc="CLAP text embed"):
        e = clap_model.get_text_embedding(texts[i:i+batch_size], use_tensor=True)
        outs.append(e.cpu().numpy())
    return np.concatenate(outs, axis=0) if outs else np.zeros((0,512), dtype=np.float32)

# ----------------------------- VGGish (FAD) -------------------------
#  : برای جلوگیری از خطای "tensors on different devices"، VGGish را کامل روی CPU می‌گذاریم.
from torchvggish import vggish, vggish_input
VGGISH_DEVICE = "cpu"
vggish_model = vggish().to(VGGISH_DEVICE).eval()

def _ensure_vggish_batch(ex):
    ex = np.asarray(ex)
    if ex.ndim == 3:   x = torch.tensor(ex, dtype=torch.float32).unsqueeze(1)  # [N,1,96,64]
    elif ex.ndim == 4: x = torch.tensor(ex, dtype=torch.float32)               # [N,1,96,64]
    else:              x = torch.zeros((1,1,96,64), dtype=torch.float32)
    return x

@torch.no_grad()
def vggish_embed_wav(path: str, sr_target=16000) -> np.ndarray:
    y  = load_mono(path, sr_target)
    ex = vggish_input.waveform_to_examples(y, sr_target)
    x  = _ensure_vggish_batch(ex).to(VGGISH_DEVICE)
    out = vggish_model(x)  # [N,128]
    return out.detach().cpu().numpy()

def fad_between_sets(paths_A: List[str], paths_B: List[str]) -> float:
    A, B = [], []
    for p in tqdm(paths_A, desc="VGGish A"):
        try:  A.append(vggish_embed_wav(p))
        except Exception as e: print("[warn VGGish A]", p, e)
    for p in tqdm(paths_B, desc="VGGish B"):
        try:  B.append(vggish_embed_wav(p))
        except Exception as e: print("[warn VGGish B]", p, e)
    if not len(A) or not len(B): return float("nan")
    A = np.concatenate(A, axis=0); B = np.concatenate(B, axis=0)
    muA, muB = A.mean(0), B.mean(0)
    covA, covB = _robust_cov(A), _robust_cov(B)
    return frechet_distance(muA, covA, muB, covB)

# ----------------------------- جمع‌آوری فایل‌ها -----------------------------
gen_files = list_audio_files(GEN_DIR)
assert len(gen_files), f"No audio found under {GEN_DIR}"

ref_files = list_audio_files(REF_DIR)
if FORCE_SELF_REF and not ref_files:
    ref_files = gen_files[:]  # self-ref → FAD/KAD ≈ 0 (ولی null نمی‌شود)

meta_df = None
if META_CSV and os.path.exists(META_CSV) and os.path.getsize(META_CSV) > 5:
    try:
        meta_df = pd.read_csv(META_CSV)
        meta_df["file_base"] = meta_df["file"].apply(lambda x: os.path.basename(str(x)))
    except Exception as e:
        print("[meta] ignored:", e); meta_df = None

# ----------------------------- امبدینگ‌ها -----------------------------
AUDIO_EMB = embed_audio_clap(gen_files)   # [N,512]
TEXTS, TEXT_EMB = None, None
if meta_df is not None and "text" in meta_df.columns:
    base2text = {r["file_base"]: str(r["text"]) for _, r in meta_df.iterrows()}
    TEXTS = [ base2text.get(os.path.basename(p), "") for p in gen_files ]
    TEXT_EMB = embed_text_clap(TEXTS)

# ----------------------------- محاسبهٔ متریک‌ها -----------------------------
metrics = {}

# Retrieval / CLAPScore
if TEXT_EMB is not None and len(TEXT_EMB) == len(AUDIO_EMB) and len(TEXT_EMB):
    A = TEXT_EMB / np.linalg.norm(TEXT_EMB, axis=1, keepdims=True)
    B = AUDIO_EMB / np.linalg.norm(AUDIO_EMB, axis=1, keepdims=True)
    S = A @ B.T
    gold = list(range(S.shape[0]))
    ranks, top1 = [], 0
    for i in range(S.shape[0]):
        order = np.argsort(-S[i])
        r = int(np.where(order == gold[i])[0][0]) + 1
        ranks.append(r); top1 += int(r==1)
    diag = S[range(len(gold)), gold]
    metrics.update({
        "R@1": round(top1/len(ranks),4),
        "MeanRank": round(float(np.mean(ranks)),2),
        "CLAPScore_mean": round(float(np.mean(diag)),4),
        "CLAPScore_median": round(float(np.median(diag)),4),
    })
else:
    metrics.update({"R@1":None,"MeanRank":None,"CLAPScore_mean":None,"CLAPScore_median":None})

# FAD(VGGish) — با CPU پایدار و غیر null وقتی ref_files موجود باشد
metrics["FAD_VGGish"] = float(fad_between_sets(gen_files, ref_files)) if len(ref_files) else None

# KAD(CLAP) / FAD(CLAP)
if len(ref_files):
    REF_EMB = embed_audio_clap(ref_files)

    def kid_unbiased_or_biased(X, Y, degree=3, gamma=None, coef=1.0, n_subsets=30, subset_size=512, seed=0):
        m, n = X.shape[0], Y.shape[0]
        subset_size = min(subset_size, m, n)
        def poly_kernel(A, B):
            d = A.shape[1]; g = (1.0/d) if (gamma is None) else gamma
            return (g*(A@B.T) + coef)**degree
        if subset_size >= 2:
            rng = np.random.default_rng(seed)
            vals=[]
            for _ in range(n_subsets):
                Xi = X[rng.choice(m, subset_size, replace=False)]
                Yi = Y[rng.choice(n, subset_size, replace=False)]
                Kxx = poly_kernel(Xi, Xi); Kyy = poly_kernel(Yi, Yi); Kxy = poly_kernel(Xi, Yi)
                np.fill_diagonal(Kxx,0.0); np.fill_diagonal(Kyy,0.0)
                mmd2 = (Kxx.sum()/(subset_size*(subset_size-1))
                        +Kyy.sum()/(subset_size*(subset_size-1))
                        -2.0*Kxy.mean())
                vals.append(mmd2)
            return float(np.mean(vals)), float(np.std(vals))
        else:
            # بایاس‌دار (فقط برای جلوگیری از null روی نمونه‌های خیلی کم)
            Kxx = poly_kernel(X, X); Kyy = poly_kernel(Y, Y); Kxy = poly_kernel(X, Y)
            np.fill_diagonal(Kxx,0.0); np.fill_diagonal(Kyy,0.0)
            mmd2 = (Kxx.mean() + Kyy.mean() - 2.0*Kxy.mean())
            return float(mmd2), 0.0

    kid_mean, kid_std = kid_unbiased_or_biased(AUDIO_EMB, REF_EMB, n_subsets=30,
                                               subset_size=min(512, AUDIO_EMB.shape[0], REF_EMB.shape[0]))
    metrics["KAD(CLAP)_mean"] = kid_mean
    metrics["KAD(CLAP)_std"]  = kid_std

    muA, muB = AUDIO_EMB.mean(0), REF_EMB.mean(0)
    covA, covB = _robust_cov(AUDIO_EMB), _robust_cov(REF_EMB)
    metrics["FAD(CLAP)"] = frechet_distance(muA, covA, muB, covB)
else:
    metrics.update({"KAD(CLAP)_mean":None,"KAD(CLAP)_std":None,"FAD(CLAP)":None})

# Diversity
if AUDIO_EMB.shape[0] >= 2:
    B = AUDIO_EMB / np.linalg.norm(AUDIO_EMB, axis=1, keepdims=True)
    P = B @ B.T
    div = 1.0 - ((P.sum() - np.trace(P)) / (P.size - P.shape[0]))
    metrics["Diversity(CLAP)"] = float(div)
else:
    metrics["Diversity(CLAP)"] = 0.0  # برای پرهیز از null (ولی علمی‌اش با یک ترک معنی ندارد)

# ----------------------------- آنالیز per-file -----------------------------
def estimate_tempo(y, sr):
    try:
        ts = librosa.beat.tempo(y=y, sr=sr, aggregate=None)
        return float(np.median(ts)) if ts is not None and len(ts) else float("nan")
    except: return float("nan")

def estimate_key(y, sr):
    try:
        chroma = librosa.feature.chroma_cqt(y=y, sr=sr)
        return int(chroma.mean(axis=1).argmax())
    except: return -1

rows=[]
for i,p in enumerate(gen_files):
    y = load_mono(p, 48000)
    tempo = estimate_tempo(y, 48000)
    key_i = estimate_key(y, 48000)
    row = {"file": p,
           "tempo_est": None if math.isnan(tempo) else round(float(tempo),2),
           "key_index_est": int(key_i) if key_i>=0 else None}
    if 'TEXT_EMB' in locals() and TEXT_EMB is not None and len(TEXT_EMB)==len(AUDIO_EMB):
        row["text"] = TEXTS[i]
        row["CLAP_text_audio"] = float(cosine_sim(TEXT_EMB[i:i+1], AUDIO_EMB[i:i+1]))
    rows.append(row)
df = pd.DataFrame(rows)

# ----------------------------- MOS (فیک در صورت نیاز) ----------------------
MOS_CSV = None
if AUTO_FAKE_MOS:
    rng = np.random.default_rng(42)
    mos_cols = [f"mos_{j+1}" for j in range(N_FAKE_RATERS)]
    tab=[]
    for p in gen_files:
        base = os.path.basename(p).lower()
        if "happy" in base: scores = rng.integers(4,6,N_FAKE_RATERS)   # 4..5
        elif "angry" in base: scores = rng.integers(3,6,N_FAKE_RATERS) # 3..5
        else: scores = rng.integers(2,6,N_FAKE_RATERS)                 # 2..5
        tab.append({"file": p, **{c:int(s) for c,s in zip(mos_cols, scores)}})
    mos_df = pd.DataFrame(tab)
    MOS_CSV = os.path.join(OUT_DIR,"mos_fake.csv"); mos_df.to_csv(MOS_CSV, index=False)

if MOS_CSV and os.path.exists(MOS_CSV):
    mos_df = pd.read_csv(MOS_CSV)
    cols = [c for c in mos_df.columns if c.lower().startswith("mos")]
    v = mos_df[cols].values.astype(float).reshape(-1)
    v = v[~np.isnan(v)]
    metrics["MOS_mean"] = float(np.mean(v)) if v.size else None
    metrics["MOS_std"]  = float(np.std(v))  if v.size else None
else:
    metrics["MOS_mean"] = metrics["MOS_std"] = None

# ----------------------------- ذخیره خروجی‌ها ------------------------------
summary_json = {"n_generated": len(gen_files),
                "n_reference": len(ref_files),
                "metrics": {k:(None if (isinstance(v,float) and (math.isnan(v) or math.isinf(v))) else v)
                            for k,v in metrics.items()}}

with open(os.path.join(OUT_DIR,"summary.json"),"w",encoding="utf-8") as f:
    json.dump(summary_json, f, indent=2, ensure_ascii=False)

df.to_csv(os.path.join(OUT_DIR,"per_file.csv"), index=False)

html_path = os.path.join(OUT_DIR,"report.html")
with open(html_path,"w",encoding="utf-8") as f:
    f.write("<h2>Audio Generation Evaluation</h2>")
    f.write("<pre>"+json.dumps(summary_json, indent=2, ensure_ascii=False)+"</pre>")
    f.write("<h3>Per-file table</h3>")
    f.write(df.to_html(index=False))
    if MOS_CSV and os.path.exists(MOS_CSV):
        f.write("<h3>MOS (fake) table</h3>")
        f.write(pd.read_csv(MOS_CSV).head(10).to_html(index=False))

zip_path = os.path.join(OUT_DIR,"eval_artifacts.zip")
with zipfile.ZipFile(zip_path,"w",zipfile.ZIP_DEFLATED) as z:
    for nm in ["summary.json","per_file.csv","report.html"]:
        z.write(os.path.join(OUT_DIR,nm), arcname=nm)
    if MOS_CSV and os.path.exists(MOS_CSV):
        z.write(MOS_CSV, arcname=os.path.basename(MOS_CSV))

print("\n=== Done. Key outputs ===")
print("Summary JSON :", os.path.join(OUT_DIR,"summary.json"))
print("Per-file CSV :", os.path.join(OUT_DIR,"per_file.csv"))
print("HTML report  :", html_path)
print("ZIP          :", zip_path)

from tabulate import tabulate
print("\nPer-file preview:\n", tabulate(df.head(20), headers="keys", tablefmt="github"))
# ===================== END ===================================================


### 3- فلسک/انگروک | وب سرویس رایگان

In [ ]:
!pip -q install flask pyngrok==7.1.6

import os
from pyngrok import ngrok, conf

# توکن من
os.environ["NGROK_AUTH_TOKEN"] = "31Wox7UWd5iAxe2bAz6BpH6jGXD_2BKkUgFYp9SE8RnHN6d7u"

# pyngrok معرفی
conf.get_default().auth_token = os.environ["NGROK_AUTH_TOKEN"]

# منطقه‌ی نزدیک‌تر برای پینگ بهتر: us, eu, ap, au, in, jp, sa
conf.get_default().region = "eu"

# اگر قبلاً تونلی باز مانده
ngrok.kill()
print("ngrok auth set.")


In [ ]:
# ===================== Flask + ngrok UI (پالیش شده) =====================
# (فرض: analyze_and_make_music ، SF2_BANKS ، SF2_PATH_DEFAULT از قبل هست)

import os, json, time
from PIL import Image
from flask import Flask, request, render_template_string, send_from_directory, jsonify, redirect, url_for
from werkzeug.utils import secure_filename
from pyngrok import ngrok, conf

# ---- NGROK auth
if "NGROK_AUTH_TOKEN" in os.environ and os.environ["NGROK_AUTH_TOKEN"].strip():
    conf.get_default().auth_token = os.environ["NGROK_AUTH_TOKEN"]
# منطقه نزدیک‌تر (): "eu","us","ap","au","in","jp","sa"
if "NGROK_REGION" in os.environ and os.environ["NGROK_REGION"].strip():
    conf.get_default().region = os.environ["NGROK_REGION"].strip()

# مسیرها
BASE_DIR   = "/content"
UPLOAD_DIR = f"{BASE_DIR}/flask_uploads"
STATIC_DIR = f"{BASE_DIR}/flask_static"
os.makedirs(UPLOAD_DIR, exist_ok=True)
os.makedirs(STATIC_DIR, exist_ok=True)

# توضیح هر SoundFont : https://musical-artifacts.com/
SF2_DESCRIPTIONS = {
    "fluidr3":     ("FluidR3_GM.sf2", "Classic GM baseline / مبنای کلاسیک"),
    "generaluser": ("GeneralUser.sf2", "Versatile GM — good all-rounder / چندمنظوره و همه‌کاره"),
    "chorium":     ("ChoriumRevA.sf2", "Bright orchestral/choirs / ارکستر/کُر درخشان"),
    "weeds":       ("WeedsGM3.sf2", "Lightweight balanced GM / سبک و متعادل"),
    "arachno":     ("Arachno v1.0.sf2", "Pop/Rock focus (guitars) / مناسب پاپ/راک"),
    "persa":       ("Persa.sf2", "Persian instruments / سازهای ایرانی"),
}

# ===== i18n (fa/en) =====
def get_texts(lang="fa"):
    fa = {
        "app_title": "موزیو | خالقِ موسیقیِ احساساتی",
        "app_sub": "متن/تصویر/صوت بده؛ موسیقیش را بساز، ارزیابی و دانلود کن.",
        "text_label": "متن (فارسی/انگلیسی)",
        "image_label": "تصویر چهره",
        "audio_label": "فایل صوتی (فرمت: wav/mp3)",
        "thr_label": "آستانه خنثی سازی احساس",
        "mins_label": "مدت موسیقی (دقیقه)",
        "seed_label": "Seed (اختیاری)",
        "style_label": "سبک / فضا",
        "style_global": "جهانی",
        "style_persian": "ایرانی",
        "sf2_builtin_label": " افکت های آماده .sf2 داخلی",
        "sf2_builtin_default": "(پیش‌فرض سیستم)",
        "sf2_upload_label": "آپلود افکت .sf2 (اختیاری)",
        "cta": "🔎 تحلیل و ساخت موسیقی",
        "result": "نتیجه",
        "download_file": "⬇️ دانلود فایل",
        "eval_report": "📊 گزارش ارزیابی",
        "sf2_used": "SoundFont",
        "detected_emotion": "احساس",
        "confidence": "اعتماد",
        "help_title": "راهنمای استفاده",
        "help_intro": "سه نوع ورودی داریم: متن یا تصویر یا صوت.(اولویت با متن است.) اگر هیچی داده نشود، خروجی خنثی می‌سازد.",
        "help_sf2": "می‌توانید از SoundFont داخلی انتخاب کنید یا فایل .sf2 خودتان را آپلود کنید.",
        "help_style": "انتخاب سبک جهانی برای سازبندی و فضای غربی است اما سبک ایرانی برای سازبندی سنتی و فضایی شرقی می‌باشد.",
        "help_eval": "در آخر، امتیازهای ساختاری و پیش‌بینی احساس روی موسیقی را می‌بینید. می‌توانید JSON ارزیابی را دانلود کنید.",
        "badge_sf2_guide": "کاربرد هر افکت",
        "lang_switch": "English",
        "json_download": "⬇️ دانلود گزارش (.JSON)",
        "labels_hint": "راهنما",
    }
    en = {
        "app_title": "Musio | Creator of Emotional Music",
        "app_sub": "Provide text/image/audio; generate music, evaluate, and download.",
        "text_label": "Text (FA/EN)",
        "image_label": "Face image",
        "audio_label": "Audio file (Format: wav/mp3)",
        "thr_label": "Emotion neutralizer threshold",
        "mins_label": "Music length (minutes)",
        "seed_label": "Seed (optional)",
        "style_label": "Style / Palette",
        "style_global": "Global",
        "style_persian": "Persian",
        "sf2_builtin_label": "Built-in SoundFont",
        "sf2_builtin_default": "(default)",
        "sf2_upload_label": "Upload SoundFont.sf2 (optional)",
        "cta": "🔎 Analyze & Generate",
        "result": "Result",
        "download_file": "⬇️ Download file",
        "eval_report": "📊 Evaluation Report",
        "sf2_used": "SoundFont",
        "detected_emotion": "Emotion",
        "confidence": "Confidence",
        "help_title": "How to use",
        "help_intro": "Provide either text OR image OR audio.(Text has priority.) If all empty, a neutral track is generated.",
        "help_sf2": "Pick a built-in SoundFont or upload your own .sf2.",
        "help_style": "Use Global for western palette, Persian for Iranian instruments (santur/ney/kamancheh/oud/...).",
        "help_eval": "Below you’ll see structural scores and an audio emotion prediction. You can also download the JSON report.",
        "badge_sf2_guide": "Application of each SoundFonts",
        "lang_switch": "فارسی",
        "json_download": "⬇️ Download report (JSON)",
        "labels_hint": "Cheat-sheet",
    }
    return (fa if lang=="fa" else en), ("rtl" if lang=="fa" else "ltr")

# ====== کمکی: گرفتن پارامترها و صدا زدن تابع اصلی ======
def _handle_generation(form, files):
    text = form.get("text","")
    threshold = float(form.get("threshold","0.7") or 0.7)
    minutes = float(form.get("minutes","3.0") or 3.0)
    seed = form.get("seed","").strip()
    seed_val = None if seed=="" else int(seed)
    style_choice = form.get("style","Global/Western")
    sf2_builtin_key = form.get("sf2_builtin","")
    sf2_path = None

    # SoundFont داخلی
    if sf2_builtin_key and sf2_builtin_key != "default":
        try:
            sf2_path = SF2_BANKS.get(sf2_builtin_key)
        except:
            sf2_path = None

    # آپلود .sf2 (مقدم بر داخلی)
    sf2_file = files.get("sf2_file")
    if sf2_file and sf2_file.filename:
        sf2_name = secure_filename(sf2_file.filename)
        sf2_path = os.path.join(UPLOAD_DIR, sf2_name)
        sf2_file.save(sf2_path)

    # تصویر
    img = None
    img_file = files.get("image")
    if img_file and img_file.filename:
        try:
            img = Image.open(img_file.stream).convert("RGB")
        except:
            img = None

    # صوت
    audio_path = None
    aud_file = files.get("audio")
    if aud_file and aud_file.filename:
        aud_name = secure_filename(aud_file.filename)
        audio_path = os.path.join(UPLOAD_DIR, aud_name)
        aud_file.save(audio_path)

    # فراخوانی همان تابع قبلی
    label, score, emoji, audio_np, report, downloadable = analyze_and_make_music(
        text, img, audio_path, threshold, minutes, seed_val, style_choice, sf2_path
    )

    # فایل خروجی را به Static کپی کنیم
    serve_name = None
    if downloadable and os.path.exists(downloadable):
        serve_name = os.path.basename(downloadable)
        dst = os.path.join(STATIC_DIR, serve_name)
        if downloadable != dst:
            try:
                import shutil; shutil.copy2(downloadable, dst)
            except Exception as e:
                print("[serve copy err]", e)

    # ذخیره JSON گزارش برای دانلود
    ts = int(time.time())
    report_name = f"report_{ts}.json"
    report_path = os.path.join(STATIC_DIR, report_name)
    try:
        with open(report_path, "w", encoding="utf-8") as f:
            json.dump(report, f, ensure_ascii=False, indent=2)
    except Exception as e:
        print("[report save err]", e)
        report_name = None

    return {
        "label": label,
        "score": float(score),
        "emoji": emoji,
        "report": report,
        "file_name": serve_name,
        "report_file": report_name,   # برای دانلود
        "soundfont": (sf2_path or SF2_PATH_DEFAULT),
        "sf2_key": sf2_builtin_key if sf2_builtin_key else "default"
    }

# =============== Flask app ===============
app = Flask(__name__)

app.jinja_env.globals.update(min=min, max=max) # متغیر غیرتعریف‌شده در قالب (Jinja2)

PAGE = r"""
<!doctype html>
<html lang="{{ 'fa' if lang_dir=='rtl' else 'en' }}" dir="{{lang_dir}}">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width,initial-scale=1" />
  <title>{{T['app_title']}}</title>
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap{{'.rtl' if lang_dir=='rtl' else ''}}.min.css" rel="stylesheet">
  <link href="https://fonts.googleapis.com/css2?family=Vazirmatn:wght@300;400;600;800&display=swap" rel="stylesheet">
  <script src="https://unpkg.com/wavesurfer.js@7"></script>
  <style>
    :root{
      --bg:#0b0e1e; --card:#121533; --fg:#eef1ff; --muted:#a7adcf; --acc:#8e7dff; --acc2:#22d3ee;
      --label:#d8dcff; --chip:#1f2547; --chip-border:#2e3566; --good:#22d3ee; --warn:#f6c065; --input-bg:#0f1230; --border:rgba(255,255,255,.12);
    }
    body{background:radial-gradient(1200px 800px at 10% 10%, #151a45 0%, transparent 60%),
                     radial-gradient(1000px 700px at 90% 20%, #1a1d4f 0%, transparent 60%),
                     linear-gradient(160deg,#0a0c1f, #0b0e1e); color:var(--fg); font-family:"Vazirmatn",system-ui,-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto;}
    .container-narrow{max-width:1024px;margin:auto;padding:22px;}
    .glass{background:rgba(255,255,255,.06);backdrop-filter: blur(10px);border:1px solid var(--border); border-radius:18px;}
    .title{font-weight:800;letter-spacing:-.02em}
    .sub{color:var(--muted)}
    .btn-gradient{background:linear-gradient(90deg,var(--acc) 0%, var(--acc2) 100%);border:none; color:#0c0f22;}
    .btn-gradient:hover{filter:brightness(1.05)}
    .btn-primary{background:linear-gradient(90deg,var(--acc),var(--acc2));border:none;border-radius:14px;padding:12px 16px;font-weight:700}
    .btn-primary:hover{filter:brightness(1.05)}
    .card{background:var(--card);border:none;border-radius:18px}
    .badge-soft{background:rgba(155,135,245,.15);color:#d0c9ff;border:1px solid rgba(155,135,245,.25)}
    .form-label{color:var(--label);font-weight:600}
    .form-text{color:var(--muted)}
    .form-control,.form-select{background:var(--input-bg) !important;color:#e9ebff !important;border:1px solid var(--border) !important;border-radius:12px}
    .form-control::placeholder{color:#c7cbea !important;opacity:.7}
    input,select,textarea{background:#0f1433!important;color:#eef1ff!important;border:1px solid rgba(255,255,255,.14)!important}
    .footer{color:#9aa0aa;font-size:.9rem}
    code.json{white-space:pre-wrap;background:#0c0f22;border-radius:12px;display:block;padding:16px;border:1px solid rgba(255,255,255,.08); direction:ltr; text-align:left;}
    a.link{color:var(--acc2);text-decoration:none}
    /* نتیجه */
    .chip{background:var(--chip); border:1px solid var(--chip-border); border-radius:12px; padding:.4rem .6rem; display:inline-flex; gap:.5rem; align-items:center;}
    .emoji{font-size:34px; line-height:1; margin-inline-start:12px;}
    .meter{height:8px; background:#0d1330; border-radius:999px; overflow:hidden}
    .meter>span{display:block; height:100%; background:linear-gradient(90deg,var(--acc),var(--acc2));}
    /* موج‌نگار */
    .wave-wrap{display:flex; align-items:center; gap:14px}
    .wave{width:100%; height:78px; border-radius:12px; background:#0d1538; border:1px solid rgba(255,255,255,.08)}
    .ctrl-btn{border:none; background:#0f1536; color:#cfe3ff; width:44px; height:44px; border-radius:12px}
    .ctrl-btn:hover{filter:brightness(1.1)}
    .vol-wrap{display:flex; align-items:center; gap:8px}
    input[type=range].vol{width:120px}
    .pill{border-radius:999px; background:#0e1331; border:1px solid rgba(255,255,255,.12); padding:.25rem .6rem; color:#cfe3ff;}
    /* آکاردئون راهنما */
    .accordion-button{background:#0f1433; color:var(--fg); border:none}
    .accordion-body{background:#0f1433; color:var(--fg)}
    /* زبان‌برگردان */
    .lang-switch a{color:#cfe3ff;text-decoration:none;padding:6px 12px;border:1px solid var(--border);border-radius:10px}
    .lang-switch a:hover{background:rgba(255,255,255,.06)}
  </style>
</head>
<body>
<div class="container-narrow">
  <div class="d-flex justify-content-between align-items-center mb-3">
    <div class="lang-switch">
      {% set switch_lang = 'en' if lang_dir=='rtl' else 'fa' %}
      <a class="link" href="{{ url_for('home') }}?lang={{switch_lang}}">{{T['lang_switch']}}</a>
    </div>
  </div>

  <div class="p-4 glass mb-4">
    <h2 class="title">🎵 🎭 {{T['app_title']}}</h2>
    <p class="sub mb-0">{{T['app_sub']}}</p>
  </div>

  <!-- راهنمای استفاده -->
  <div class="accordion mb-4" id="helpAcc">
    <div class="accordion-item">
      <h2 class="accordion-header">
        <button class="accordion-button collapsed" type="button" data-bs-toggle="collapse" data-bs-target="#help">
          ℹ️ {{T['help_title']}}
        </button>
      </h2>
      <div id="help" class="accordion-collapse collapse" data-bs-parent="#helpAcc">
        <div class="accordion-body">
          <ul>
            <li>{{T['help_intro']}}</li>
            <li>{{T['help_sf2']}}</li>
            <li>{{T['help_style']}}</li>
            <li>{{T['help_eval']}}</li>
          </ul>
        </div>
      </div>
    </div>
  </div>

  <div class="card p-4 mb-4">
    <form action="{{ url_for('generate') }}?lang={{'fa' if lang_dir=='rtl' else 'en'}}" method="post" enctype="multipart/form-data">
      <div class="row g-3">
        <div class="col-12">
          <label class="form-label">{{T['text_label']}}</label>
          <textarea name="text" class="form-control" rows="3" placeholder="{{ 'مثال: امروز خیلی خوشحالم!' if lang_dir=='rtl' else 'e.g., I feel super happy today!' }}"></textarea>
        </div>
        <div class="col-md-6">
          <label class="form-label">{{T['image_label']}}</label>
          <input type="file" name="image" accept="image/*" class="form-control" />
        </div>
        <div class="col-md-6">
          <label class="form-label">{{T['audio_label']}}</label>
          <input type="file" name="audio" accept=".wav,.mp3,.flac,.ogg,.m4a" class="form-control" />
        </div>

        <div class="col-md-3">
          <label class="form-label">{{T['thr_label']}}</label>
          <input type="number" step="0.01" min="0" max="1" name="threshold" value="0.7" class="form-control" />
        </div>
        <div class="col-md-3">
          <label class="form-label">{{T['mins_label']}}</label>
          <input type="number" step="0.5" min="1" max="6" name="minutes" value="3.0" class="form-control" />
        </div>
        <div class="col-md-3">
          <label class="form-label">{{T['seed_label']}}</label>
          <input type="text" name="seed" class="form-control" placeholder="{{ 'خالی = تصادفی' if lang_dir=='rtl' else 'Empty = random' }}" />
        </div>
        <div class="col-md-3">
          <label class="form-label">{{T['style_label']}}</label>
          <select name="style" class="form-select">
            <option>{{T['style_global']}}</option>
            <option>{{T['style_persian']}}</option>
          </select>
        </div>

        <div class="col-8">
          <label class="form-label">{{T['sf2_builtin_label']}}</label>
          <select name="sf2_builtin" class="form-select">
            <option value="default">{{T['sf2_builtin_default']}}</option>
            {% for key, path in sf2_list %}
            <option value="{{key}}">{{key}} — {{path}}</option>
            {% endfor %}
          </select>
          <div class="form-text">{{ 'یا فایل .sf2 دلخواه‌ را در باکس روبرو آپلود کنید (که بر داخلی مقدم است).' if lang_dir=='rtl' else 'Or upload a .sf2 by yourself (takes precedence).' }}</div>
        </div>

        <div class="col-md-4">
          <label class="form-label">{{T['sf2_upload_label']}}</label>
          <input type="file" name="sf2_file" accept=".sf2" class="form-control" />
        </div>

        <!-- راهنمای SoundFont -->
        <div class="accordion mb-4" id="helpsf2">
          <div class="accordion-item">
            <h3 class="accordion-header">
              <button class="accordion-button collapsed" type="button" data-bs-toggle="collapse" data-bs-target="#helps">
                ℹ️ {{T['badge_sf2_guide']}}
              </button>
            </h3>
            <div id="helps" class="accordion-collapse collapse" data-bs-parent="#helpsf2">
              <div class="accordion-body">
                <ul>
                  <div class="col-12">
                    <div class="mt-2 d-flex flex-wrap gap-2">
                      {% for k, info in sf2_desc %}
                        <span class="pill">{{info[0]}} || {{info[1]}}</span>
                      {% endfor %}
                    </div>
                  </div>
                </ul>
              </div>
            </div>
          </div>
        </div>

        <div class="col-12 d-grid mt-4">
          <button class="btn btn-primary btn-lg">{{T['cta']}}</button>
        </div>
      </div>
    </form>
  </div>

  {% if result %}
  <div class="card p-4 mb-4">
    <div class="form-label d-flex justify-content-between align-items-center flex-wrap gap-2">
      <h5 class="mb-0">{{T['result']}}</h5>
      <span class="pill">{{T['sf2_used']}}: {{result.sf2_title}}</span>
    </div>
    <hr>
    <div class="row g-3 align-items-center">
      <div class="col-12 col-lg-8">
        <!-- موج‌نگار + کنترل‌ها -->
        <div class="wave-wrap">
          <button class="ctrl-btn" id="btnFwd" title="+10s">⏩</button>
          <button class="ctrl-btn" id="btnPlay" title="Play/Pause">▶️</button>
          <button class="ctrl-btn" id="btnBack" title="−10s">⏪</button>
          <div class="vol-wrap">
            🔊<input type="range" min="0" max="1" value="0.9" step="0.01" class="vol" id="vol">
          </div>
          <div id="waveform" class="wave" data-audio-url="{{ '/file/' + result.file_name }}"></div>
        </div>
        <div class="mt-2">
          {% if result.file_name %}
            <a class="btn btn-outline-info btn-sm" href="/file/{{result.file_name}}" download>{{T['download_file']}}</a>
          {% endif %}
        </div>
      </div>
      <div class="col-12 col-lg-4">
        <!-- ایموجی بدون باکس + چیپ‌ها -->
        <div class="d-flex align-items-center">
          <div class="emoji">{{result.emoji}}</div>
          <div class="form-label ms-3">
            <div class="chip">
              <strong>{{T['detected_emotion']}}:</strong>
              <span>{{result.label}}</span>
            </div>
            <div class="form-label mt-2">
              <div class="form-label d-flex justify-content-between"><small class="form-label">{{T['confidence']}}</small><small>{{'%.0f' % (result.score*100)}}%</small></div>
              <div class="meter mt-1"><span style="width: {{ max(4, min(100, result.score*100)) }}%"></span></div>
            </div>
          </div>
        </div>
        <!-- جای توضیح کوتاه -->
        <div class="form-label mt-3 small text-muted">
          <span class="badge bg-secondary">{{T['labels_hint']}}</span>
          <div class="form-label mt-1">
            {{ 'Global → پاپ/راک/الکترونیک، Persian → سنتی/محلی. برای شاد: Arachno/Weeds، برای ارکستر: Chorium، برای ایرانی: Persa.'
               if lang_dir=='rtl' else
               'Global → pop/rock/electronic, Persian → traditional Iranian. For happy: Arachno/Weeds, for orchestral: Chorium, for Iranian: Persa.' }}
          </div>
        </div>
      </div>
    </div>

    <hr class="mt-4">
    <h6 class="form-label mb-2">{{T['eval_report']}}</h6>
    <code class="json">{{result.report_json}}</code>
    <div class="d-flex gap-2 mb-2">
      {% if result.report_file %}
        <a class="btn btn-outline-info btn-sm" href="/file/{{result.report_file}}" download>{{T['json_download']}}</a>
      {% endif %}
    </div>
  </div>
  {% endif %}

  <div class="footer text-center py-3">
    Flask + ngrok · Wavesurfer.js · Bootstrap || {{ 'برای مهندسانِ هنرمند ✨' if lang_dir=='rtl' else 'crafted for Artist Engineers ✨' }}
  </div>
</div>

<script>
(function(){
  // Wavesurfer
  const el = document.getElementById('waveform');
  if(!el) return;
  const url = el.getAttribute('data-audio-url');
  if(!url){ return; }
  const ws = WaveSurfer.create({
    container: el,
    waveColor: '#5f6ee3',
    progressColor: '#22d3ee',
    cursorColor: '#ffffff',
    height: 78,
    barWidth: 2,
    barRadius: 2,
    barGap: 2,
    normalize: true,
    hideScrollbar: true,
  });
  ws.load(url);
  // Controls
  const btnPlay = document.getElementById('btnPlay');
  const btnBack = document.getElementById('btnBack');
  const btnFwd  = document.getElementById('btnFwd');
  const vol     = document.getElementById('vol');
  if(btnPlay){ btnPlay.onclick = ()=>{ ws.playPause(); btnPlay.textContent = ws.isPlaying() ? '⏸️' : '▶️'; } }
  if(btnBack){ btnBack.onclick = ()=> ws.skip(-10) }
  if(btnFwd){  btnFwd.onclick  = ()=> ws.skip(10) }
  if(vol){     vol.oninput     = (e)=> ws.setVolume(parseFloat(e.target.value)); }
  ws.on('play', ()=> btnPlay && (btnPlay.textContent='⏸️'));
  ws.on('pause',()=> btnPlay && (btnPlay.textContent='▶️'));
})();
</script>

<script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/js/bootstrap.bundle.min.js"></script>
</body>
</html>
"""

@app.route("/", methods=["GET"])
def home():
    lang = request.args.get("lang","fa").lower()
    T, lang_dir = get_texts(lang)
    try:
        sf2_items = list(SF2_BANKS.items())
    except:
        sf2_items = []
    sf2_desc = list(SF2_DESCRIPTIONS.items())
    return render_template_string(PAGE, T=T, lang_dir=lang_dir, sf2_list=sf2_items, sf2_desc=sf2_desc, result=None)

@app.route("/generate", methods=["POST"])
def generate():
    lang = request.args.get("lang","fa").lower()
    T, lang_dir = get_texts(lang)
    result = _handle_generation(request.form, request.files)

    # آماده‌سازی عنوان SF2
    key = result.get("sf2_key","default")
    if key in SF2_DESCRIPTIONS:
        sf2_title = SF2_DESCRIPTIONS[key][0]
    else:
        # اگر آپلودی بوده یا default
        sf2_title = os.path.basename(result["soundfont"])

    # URL فایل صوتی
    file_url = None
    if result["file_name"]:
        file_url = request.host_url.rstrip("/") + "/file/" + result["file_name"]

    res_for_view = {
        "label": result["label"],
        "score": result["score"],
        "emoji": result["emoji"],
        "file_name": result["file_name"],
        "report_file": result["report_file"],
        "soundfont": result["soundfont"],
        "sf2_title": sf2_title,
        "report_json": json.dumps(result["report"], ensure_ascii=False, indent=2)
    }

    try:
        sf2_items = list(SF2_BANKS.items())
    except:
        sf2_items = []
    sf2_desc = list(SF2_DESCRIPTIONS.items())
    return render_template_string(
        PAGE, T=T, lang_dir=lang_dir, sf2_list=sf2_items, sf2_desc=sf2_desc,
        result=res_for_view, file_url=file_url
    )

@app.route("/file/<path:fname>")
def serve_file(fname):
    return send_from_directory(STATIC_DIR, fname, as_attachment=False)

# ===== launch with ngrok (اگر قبلاً پورت اشغال است، پورت جدید) =====
try:
    ngrok.kill()
except:
    pass

port = int(os.environ.get("FLASK_PORT", 5000))
public_url = ngrok.connect(port).public_url
print("Public URL:", public_url)

# اگر روی 5000 اشغال بود، Flask را روی پورت آزاد روشن کن
import socket
def _port_in_use(p):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(("0.0.0.0", p)) == 0

if _port_in_use(port):
    # پیدا کردن پورت آزاد
    for candidate in range(5001, 5100):
        if not _port_in_use(candidate):
            port = candidate
            break
    print("Switched to port:", port)

app.run(host="0.0.0.0", port=port, debug=False)
# ===================================================================
